In [1]:
from collections import defaultdict

import numpy as np
import re
import matplotlib.pyplot as plt

## 2.4.5
Дана следующая коллекция текстов. Постройте словарь (отображение из строкового представления токенов в их номера) и вектор весов (DF).

$DF(w) = \frac{DocCount(w, c)}{Size(c)}$

  -- частота слова $w$ в коллекции $c$ (отношение количества документов, в которых слово используется, к общему количеству документов).

Казнить нельзя, помиловать. Нельзя наказывать.

Казнить, нельзя помиловать. Нельзя освободить.

Нельзя не помиловать.

Обязательно освободить.

При токенизации используйте регулярное выражение из семинара: [\w\d]+. После токенизации все токены нужно привести к нижнему регистру. Фильтрацию по частоте не использовать.

Ответ запишите в две строки:

в первой строке - содержимое словаря - список уникальных токенов через пробел в порядке возрастания частоты встречаемости. При одинаковой частоте сортировать по алфавиту.
во второй строке - список весов (DF) токенов, округлённых до 2 знака после запятой и разделённых пробелами, в том же порядке, что и токены в первой строке.

In [2]:
# 2.4.5
simple_regex = re.compile(r'[\w\d]+')
texts = ['Казнить нельзя, помиловать. Нельзя наказывать.',
         'Казнить, нельзя помиловать. Нельзя освободить.',
         'Нельзя не помиловать.', 
         'Обязательно освободить.']

print(simple_regex.findall(texts[0]))

def tokenize_text(text, regex=simple_regex, min_len=0):
    text = text.lower()
    all_tokens = regex.findall(text)
    return [token for token in all_tokens if len(token) > min_len]

def tokenize_corpus(texts, tokenizer=tokenize_text, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

tokenized_corpus = tokenize_corpus(texts)
print(tokenized_corpus)

word_counts = defaultdict(int)
doc_n = 0

for text in tokenized_corpus:
    doc_n += 1
    unique_tokens = set(text)
    for token in unique_tokens:
        word_counts[token] += 1
        
print("Word counts:\n", word_counts)
print("Число документов:", doc_n)

sorted_words = sorted(word_counts.items(), reverse=False, key=lambda pair: (pair[1], pair[0]))
print(sorted_words)

document_freq = [cnt / doc_n for _, cnt in sorted_words]

print(document_freq)

print(f'{" ".join([word for word, _ in sorted_words])}\n{" ".join(map(str, document_freq))}')

word2id = {token: i for i, (token, _) in enumerate(sorted_words)}
print(word2id)

['Казнить', 'нельзя', 'помиловать', 'Нельзя', 'наказывать']
[['казнить', 'нельзя', 'помиловать', 'нельзя', 'наказывать'], ['казнить', 'нельзя', 'помиловать', 'нельзя', 'освободить'], ['нельзя', 'не', 'помиловать'], ['обязательно', 'освободить']]
Word counts:
 defaultdict(<class 'int'>, {'наказывать': 1, 'казнить': 2, 'нельзя': 3, 'помиловать': 3, 'освободить': 2, 'не': 1, 'обязательно': 1})
Число документов: 4
[('наказывать', 1), ('не', 1), ('обязательно', 1), ('казнить', 2), ('освободить', 2), ('нельзя', 3), ('помиловать', 3)]
[0.25, 0.25, 0.25, 0.5, 0.5, 0.75, 0.75]
наказывать не обязательно казнить освободить нельзя помиловать
0.25 0.25 0.25 0.5 0.5 0.75 0.75
{'наказывать': 0, 'не': 1, 'обязательно': 2, 'казнить': 3, 'освободить': 4, 'нельзя': 5, 'помиловать': 6}


## 2.4.7
Постройте матрицу признаков для текстов с шага 5 с использованием словаря и вектора весов, полученного на шаге 5. Используйте взвешивание $lTFIDF = \ln(TF + 1) \cdot IDF$

Значения признаков следует отмасштабировать так, чтобы для каждого признака его среднее значение по выборке равнялось 0, а среднеквадратичное отклонение 1: $x^{scaled}_{i} = \frac{x_{i} - E(x)} {\sigma(x)}x$

В результате масштабирования для каждого столбца матрицы признаков среднее должно равняться 0, а среднеквадратичное отклонение 1.

При расчёте среднеквадратического отклонения необходимо использовать скорректированную оценку $\sigma=\sqrt{\frac{\sum_{i-1}^n(x_i - E(x))^2}{n - 1}}$
Чтобы получить такую оценку с помощью numpy, необходимо передать параметр ddof=1: 

    feature_matrix = np.zeros((num_docs, num_feats))
    feats_std = feature_matrix.std(0, ddof=1)
    
Ответ отформатируйте так, чтобы на каждой строке были признаки одного документа. Порядок столбцов должен соответствовать порядку слов в словаре (как в ответе на шаге 5, по возрастанию df). Столбцы разделяйте одним пробелом. В качестве разделителя целой и дробной части используйте точку или запятую. Округлять значения не обязательно. Решение, при проверке, автоматически округлится до двух знаков. Метод округления - либо "математический", либо свойственный Python rounding half to even strategy, если интересно, посмотрите IEEE 754.

Пример ответа для первых двух документов (до полного ответа не хватает ещё двух строк):

    1.5  -0.5 -0.5 0.87 -0.76 0.60 0.16
    -0.5 -0.5 -0.5 0.87 0.18  0.60 0.16

In [3]:
# 2.4.7

vectorized_texts = np.zeros(shape=(len(tokenized_corpus), len(word2id)), dtype='float32')
print(vectorized_texts)

for text_i, text in enumerate(tokenized_corpus):
    for token in text:
        if token in word2id:
            vectorized_texts[text_i, word2id[token]] += 1
            
print("Встречаемость слова в документе:\n", vectorized_texts)

print("Бинарный вариант:\n", (vectorized_texts > 0).astype(int))

tf_vectorized = vectorized_texts / vectorized_texts.sum(axis=1, keepdims=True)
print("TF вариант:\n", tf_vectorized)

tfidf_vectorized = tf_vectorized / document_freq
print("TF-IDF вариант:\n", tfidf_vectorized)

ltfidf_vectorized = np.log(tf_vectorized + 1) / document_freq
print("LTF-IDF вариант:\n", ltfidf_vectorized)

ltfidf_vectorized_scaled = (ltfidf_vectorized - ltfidf_vectorized.mean(axis=0)) / ltfidf_vectorized.std(axis=0, ddof=1)
print("LTF-IDF вариант стандартизированный:\n", ltfidf_vectorized_scaled)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
Встречаемость слова в документе:
 [[1. 0. 0. 1. 0. 2. 1.]
 [0. 0. 0. 1. 1. 2. 1.]
 [0. 1. 0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 1. 0. 0.]]
Бинарный вариант:
 [[1 0 0 1 0 1 1]
 [0 0 0 1 1 1 1]
 [0 1 0 0 0 1 1]
 [0 0 1 0 1 0 0]]
TF вариант:
 [[0.2        0.         0.         0.2        0.         0.4
  0.2       ]
 [0.         0.         0.         0.2        0.2        0.4
  0.2       ]
 [0.         0.33333334 0.         0.         0.         0.33333334
  0.33333334]
 [0.         0.         0.5        0.         0.5        0.
  0.        ]]
TF-IDF вариант:
 [[0.80000001 0.         0.         0.40000001 0.         0.53333334
  0.26666667]
 [0.         0.         0.         0.40000001 0.40000001 0.53333334
  0.26666667]
 [0.         1.33333337 0.         0.         0.         0.44444446
  0.44444446]
 [0.         0.         2.         0.         1.         0.
  0.        ]]
LTF-IDF вариант:
 [[0.7292

## 3.4.1
Вы обучаете Word2Vec Skip Gram Negative Sampling с окном заданной ширины. Например, окно размерности 5 подразумевает, что положительными примерами считаются слова, отстоящие от центрального слова не более чем на 2 позиции влево или вправо. Центральное слово не учитывается как контекстное слово.

Напишите функцию, которая генерирует обучающие примеры из текста. Каждый обучающий пример должен иметь вид кортежа из трёх элементов $(CenterWord, CtxWord, Label)$, где $CenterWord \in \mathbb{N}$ - идентификатор токена в центре окна, $CtxWord \in \mathbb{N}$ - идентификатор соседнего токена, $Label \in \{0, 1\}$ - 1 если $CtxWord$ настоящий и 0, если это отрицательный пример.

Функция должна возвращать список обучающих примеров.

Аргумент ns_rate задаёт количество отрицательных примеров, которое нужно сгенерировать на каждый положительный пример. При семплировании отрицательных слов обычно не проверяют, встречается ли это слово в окне. Таким образом, среди отрицательных примеров могут появиться положительные.

Входной текст уже токенизирован, токены заменены на их идентификаторы.

Тесты генерируются случайно, ограничения:

- len(text) < 20
- window_size <= 11, нечётное
- vocab_size < 100
- ns_rate < 3
Слова имеют идентификаторы 0..vocab_size - 1 (как возвращает np.random.randint).

Обратите также внимание на то, что -3 // 2 != -(3 // 2).

In [4]:
# 3.4.1

import sys
import ast
import numpy as np
from itertools import product


sample_input = (
    [1, 0, 1, 0, 0, 5, 0, 3, 5, 5, 3, 0, 5, 0, 5, 2, 0, 1, 3],
    3,
    6,
    1
)

sample_output = [
    [1, 0, 1],
    [1, 2, 0],
    [0, 1, 1], 
    [0, 1, 0], 
    [0, 1, 1], 
    [0, 2, 0],
    [1, 0, 1],
    [1, 3, 0],
    [1, 0, 1],
    [1, 2, 0], 
    [0, 1, 1], 
    [0, 1, 0],
    [0, 0, 1],
    [0, 3, 0], 
    [0, 0, 1],
    [0, 2, 0],
    [0, 5, 1],
    [0, 2, 0], 
    [5, 0, 1], 
    [5, 2, 0], 
    [5, 0, 1],
    [5, 0, 0], 
    [0, 5, 1], 
    [0, 4, 0],
    [0, 3, 1], 
    [0, 1, 0],
    [3, 0, 1],
    [3, 0, 0], 
    [3, 5, 1], 
    [3, 2, 0], 
    [5, 3, 1], 
    [5, 1, 0], 
    [5, 5, 1], 
    [5, 4, 0], 
    [5, 5, 1], 
    [5, 2, 0], 
    [5, 3, 1], 
    [5, 0, 0], 
    [3, 5, 1], 
    [3, 0, 0], 
    [3, 0, 1], 
    [3, 5, 0], 
    [0, 3, 1], 
    [0, 2, 0], 
    [0, 5, 1], 
    [0, 1, 0], 
    [5, 0, 1],
    [5, 2, 0], 
    [5, 0, 1],
    [5, 2, 0],
    [0, 5, 1],
    [0, 3, 0],
    [0, 5, 1],
    [0, 1, 0],
    [5, 0, 1],
    [5, 1, 0], 
    [5, 2, 1], 
    [5, 2, 0], 
    [2, 5, 1],
    [2, 3, 0],
    [2, 0, 1], 
    [2, 5, 0],
    [0, 2, 1], 
    [0, 2, 0],
    [0, 1, 1],
    [0, 3, 0], 
    [1, 0, 1],
    [1, 2, 0],
    [1, 3, 1],
    [1, 0, 0],
    [3, 1, 1],
    [3, 4, 0]
]

def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def make_diag_mask(size, radius):
    """Квадратная матрица размера Size x Size с двумя полосами ширины radius вдоль главной диагонали"""
    idxs = np.arange(size)
    abs_idx_diff = np.abs(np.arange(size)[np.newaxis, :] - np.arange(size)[:, np.newaxis])
    mask = ((abs_idx_diff <= radius // 2) & (abs_idx_diff > 0)).astype(int)
    return mask

def generate_w2v_sgns_samples(text, window_size, vocab_size, ns_rate):
    """
    text - list of integer numbers - ids of tokens in text
    window_size - odd integer - width of window
    vocab_size - positive integer - number of tokens in vocabulary
    ns_rate - positive integer - number of negative tokens to sample per one positive sample

    returns list of training samples (CenterWord, CtxWord, Label)
    """
    text = np.array(text)
    size = len(text)
    context_words_idxs = make_diag_mask(size, window_size)
    center_words_idx = np.eye(N=size, dtype=int)

    result = []

    for i in range(size):
        context = context_words_idxs[i]
        center = center_words_idx[i]

        context_idx = np.where(context != 0)[0]
        center_idx = np.where(center != 0)[0]

        center_word = text[center_idx]
        context_words = text[context_idx]

        positive_examples = [list(i) + [1] for i in product(center_word, context_words)]    

        # чтобы не совпадало с самим собой
#         all_words = np.arange(vocab_size)
#         center_word_idx = np.argwhere(all_words == center_word)[0][0]
#         all_words = list(all_words)
#         del all_words[center_word_idx]
#         all_words = np.array(all_words)
#         negative_word = np.random.choice(all_words)

        # выбрать одно из двух, либо убрать совпадение с самим собой, либо нет
        negative_words = np.random.randint(vocab_size, size=ns_rate*len(positive_examples))    
        negative_examples = [list(i) + [0] for i in product(center_word, negative_words)]

        positive_negative_examples = positive_examples + negative_examples    
        result.extend(positive_negative_examples)
    
    return result

result = generate_w2v_sgns_samples(*sample_input)

result

[[1, 0, 1],
 [1, 5, 0],
 [0, 1, 1],
 [0, 1, 1],
 [0, 3, 0],
 [0, 2, 0],
 [1, 0, 1],
 [1, 0, 1],
 [1, 4, 0],
 [1, 2, 0],
 [0, 1, 1],
 [0, 0, 1],
 [0, 4, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 5, 1],
 [0, 3, 0],
 [0, 0, 0],
 [5, 0, 1],
 [5, 0, 1],
 [5, 3, 0],
 [5, 1, 0],
 [0, 5, 1],
 [0, 3, 1],
 [0, 5, 0],
 [0, 2, 0],
 [3, 0, 1],
 [3, 5, 1],
 [3, 2, 0],
 [3, 3, 0],
 [5, 3, 1],
 [5, 5, 1],
 [5, 5, 0],
 [5, 5, 0],
 [5, 5, 1],
 [5, 3, 1],
 [5, 1, 0],
 [5, 3, 0],
 [3, 5, 1],
 [3, 0, 1],
 [3, 0, 0],
 [3, 1, 0],
 [0, 3, 1],
 [0, 5, 1],
 [0, 2, 0],
 [0, 1, 0],
 [5, 0, 1],
 [5, 0, 1],
 [5, 3, 0],
 [5, 4, 0],
 [0, 5, 1],
 [0, 5, 1],
 [0, 1, 0],
 [0, 0, 0],
 [5, 0, 1],
 [5, 2, 1],
 [5, 5, 0],
 [5, 5, 0],
 [2, 5, 1],
 [2, 0, 1],
 [2, 5, 0],
 [2, 4, 0],
 [0, 2, 1],
 [0, 1, 1],
 [0, 0, 0],
 [0, 2, 0],
 [1, 0, 1],
 [1, 3, 1],
 [1, 2, 0],
 [1, 0, 0],
 [3, 1, 1],
 [3, 5, 0]]

## 3.4.2

Вы обучаете Word2Vec Skip Gram Negative Sampling.

Напишите функцию, обновляющую веса модели при получении одного обучающего примера в формате $(CenterWord, CtxWord, Label)$.

При обучении предсказания модели вычисляются по формуле $P(CtxWord | CenterWord) = \sigma(W_{CenterWord, :} \cdot D_{CtxWord,:})$.

Функция потерь - бинарная кросс-энтропия.

Тесты генерируются случайно.

In [5]:
0.9836099232994968 - 0.7561584406822225, 0.3847689688960674 - 0.6340566555548147

(0.22745148261727433, -0.24928768665874734)

In [6]:
0.882545488649187 - 0.6290474670186392

0.2534980216305478

In [7]:
sample_input = (
    [
        [0.3449417709491044, 0.6762047256081501, 0.9583446027893963],
        [0.6247126159157468, 0.22038323197740317, 0.29717611444948355],
        [0.9836099232994968, 0.3847689688960674, 0.033312247867206435],
        [0.4217704869846559, 0.0023859008971685025, 0.009686915033163657],
        [0.6933070658521228, 0.9705089533296152, 0.9189360293193337],
        [0.024858486425111903, 0.11331113152689753, 0.6492144300167894],
        [0.7861289466352543, 0.227319130535791, 0.8165251907260063],
        [0.7672181161105678, 0.04865001026002924, 0.07514404284170773]
    ],
    [
        [0.4628817426583818, 0.7747296319956671, 0.1374808935513827],
        [0.17026823169513283, 0.4094733988461122, 0.3175531656197459],
        [0.2910876746161247, 0.6340566555548147, 0.23158010794029804],
        [0.8449042648180852, 0.4796593509107806, 0.11278090182290745],
        [0.049097778744511156, 0.6254116250148337, 0.13038703647472905],
        [0.882545488649187, 0.6223076699449618, 0.1633041302523962],
        [0.6704032810194875, 0.941803340812521, 0.7358646489592193],
        [0.9875878745059805, 0.17935677165390562, 0.6798846454394736]
    ],
    2,
    5,
    0,
    0.342405260598321
)

sample_output = [
    [
        [0.3449417709491044, 0.6762047256081501, 0.9583446027893963],
        [0.6247126159157468, 0.22038323197740317, 0.29717611444948355],
        [0.7561584406822225, 0.22438652516534294, -0.008774836618697823],
        [0.4217704869846559, 0.0023859008971685025, 0.009686915033163657],
        [0.6933070658521228, 0.9705089533296152, 0.9189360293193337],
        [0.024858486425111903, 0.11331113152689753, 0.6492144300167894],
        [0.7861289466352543, 0.227319130535791, 0.8165251907260063],
        [0.7672181161105678, 0.04865001026002924, 0.07514404284170773]
    ],
    [
        [0.4628817426583818, 0.7747296319956671, 0.1374808935513827],
        [0.17026823169513283, 0.4094733988461122, 0.3175531656197459],
        [0.2910876746161247, 0.6340566555548147, 0.23158010794029804],
        [0.8449042648180852, 0.4796593509107806, 0.11278090182290745],
        [0.049097778744511156, 0.6254116250148337, 0.13038703647472905],
        [0.6290474670186392, 0.5231442006778062, 0.15471882755224034],
        [0.6704032810194875, 0.941803340812521, 0.7358646489592193],
        [0.9875878745059805, 0.17935677165390562, 0.6798846454394736]
    ]
]

import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))
    
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    s = 1 / (1 + np.exp(-z))
    return s * (1 - s)

def log_loss(y_true, a_pred):
    '''
    Compute log loss, a_pred - vector of size n_objects
    '''
    return np.mean(-y_true * np.log(a_pred) - (1 - y_true) * np.log(1 - a_pred))

def log_loss_derivative(y_true, a_pred):
    '''
    Compute detivative of log loss
    '''
    return (-y_true / a_pred + (1 - y_true) / (1 - a_pred)) / len(y_true)

def update_w2v_weights(center_embeddings, context_embeddings, center_word, context_word, label, learning_rate):
    """
    center_embeddings - VocabSize x EmbSize
    context_embeddings - VocabSize x EmbSize
    center_word - int - identifier of center word
    context_word - int - identifier of context word
    label - 1 if context_word is real, 0 if it is negative
    learning_rate - float > 0 - size of gradient step
    """
    # Переведем в numpy для удобства вычислений
    center_embeddings = np.asarray(center_embeddings)
    context_embeddings = np.asarray(context_embeddings)
    
    # Выберем центральное и контекстное слово
    center_embedding = center_embeddings[center_word]
    context_embedding = context_embeddings[context_word]
    
    # Считаем "оценку" сходства и ее вероятность по формуле 𝑃(𝐶𝑡𝑥𝑊𝑜𝑟𝑑|𝐶𝑒𝑛𝑡𝑒𝑟𝑊𝑜𝑟𝑑)=𝜎(𝑊𝐶𝑒𝑛𝑡𝑒𝑟𝑊𝑜𝑟𝑑,:⋅𝐷𝐶𝑡𝑥𝑊𝑜𝑟𝑑,:)
    score = center_embedding.dot(context_embedding)
    prob = sigmoid(score)
    print(f'LOSS: {log_loss(label, prob)}')
    
    # Считаем производные для правила цепочки: производная лосса * производную сигмоиды
    # * производную 𝑊𝐶𝑒𝑛𝑡𝑒𝑟𝑊𝑜𝑟𝑑,:⋅𝐷𝐶𝑡𝑥𝑊𝑜𝑟𝑑,: по 𝑊𝐶𝑒𝑛𝑡𝑒𝑟𝑊𝑜𝑟𝑑 и 𝐷𝐶𝑡𝑥𝑊𝑜𝑟𝑑
    log_loss_deriv = log_loss_derivative(np.array([label]), np.array([prob]))
    sigmoid_deriv = sigmoid_derivative(score)
    
    # Считаем производные по цетральному слову и контекстному для обновления весов
    center_grad = log_loss_deriv * sigmoid_deriv * context_embedding
    context_grad = log_loss_deriv * sigmoid_deriv * center_embedding
    print(f'Center word grad: {center_grad}')
    print(f'Context word grad: {context_grad}')
    
    center_embeddings[center_word] -= learning_rate * center_grad
    context_embeddings[context_word] -= learning_rate * context_grad
    
    return center_embeddings, context_embeddings

result = update_w2v_weights(*sample_input)

print(np.allclose(result[0], np.array(sample_output[0])))
print(np.allclose(result[1], np.array(sample_output[1])))

LOSS: 1.3970783178926671
Center word grad: [0.66427567 0.46839947 0.122916  ]
Context word grad: [0.740345   0.28960849 0.02507351]
True
True


## 3.4.3
Вы обучаете FastText SkipGram Negative Sampling с окном заданной ширины (аналогично [первой задаче](https://stepik.org/lesson/261476/step/2?unit=242225) 3.4.1). Общий алгоритм обучения также описывался [ранее](https://stepik.org/lesson/225313/step/11?unit=198056). Единственное существенное отличие от Word2Vec - использование N-грамм для получения вектора центрального слова.

Напишите функцию, которая генерирует обучающие примеры из текста. Каждый обучающий пример должен иметь вид кортежа из трёх элементов $(CenterSubwords, CtxWord, Label)$, где CenterSubwordsCenterSubwords - список идентификаторов N-грамм, входящих в токен в центре окна (включая идентификатор самого токена), $CtxWord \in \mathbb{N}$ - идентификатор соседнего токена, $Label \in \{0, 1\}$ - 1 если CtxWordCtxWord настоящий и 0, если это отрицательный пример.

Функция должна возвращать список обучающих примеров.

Аргумент ns_rate задаёт количество отрицательных примеров, которое нужно сгенерировать на каждый положительный пример.

Входной текст уже токенизирован, токены заменены на их идентификаторы.

Для получения списка N-грамм, входящих в токен, используется заранее построенный словарь (он передаётся в функцию, которую Вам нужно написать). В этом задании следует считать, что сам токен всегда есть в словаре и поэтому нужно обновлять вектор для него (хотя при обучении FastText на настоящих данных это может быть не так). Все n-граммы имеют номера больше или равные vocab_size.

Тесты генерируются случайно, ограничения:

- len(text) < 20
- vocab_size < 10
- window_size <= 11, нечётное
- ns_rate < 3
- ngrams_n < 20
Слова имеют идентификаторы 0..vocab_size - 1 (как возвращает np.random.randint). N-граммы имеют идентификаторы vocab_size .. vocab_size + ngrams_n - 1.

Обратите также внимание на то, что -3 // 2 != -(3 // 2).

In [8]:
import sys
import ast
import numpy as np

sample_input = (
    [1, 2, 0, 1, 4, 0, 4, 1, 5, 4, 5, 4, 5, 1],
    3,
    6,
    2,
    [[17], [10, 12], [20, 20], [7, 13], [], [7, 11]]
)
sample_output = [
    ([1, 10, 12], 2, 1),
    ([1, 10, 12], 5, 0),
    ([1, 10, 12], 2, 0),
    ([2, 20], 1, 1),
    ([2, 20], 0, 0),
    ([2, 20], 5, 0),
    ([2, 20], 0, 1),
    ([2, 20], 1, 0),
    ([2, 20], 0, 0),
    ([0, 17], 2, 1),
    ([0, 17], 1, 0),
    ([0, 17], 3, 0),
    ([0, 17], 1, 1),
    ([0, 17], 1, 0),
    ([0, 17], 3, 0),
    ([1, 10, 12], 0, 1),
    ([1, 10, 12], 4, 0),
    ([1, 10, 12], 3, 0),
    ([1, 10, 12], 4, 1),
    ([1, 10, 12], 5, 0),
    ([1, 10, 12], 3, 0),
    ([4], 1, 1),
    ([4], 5, 0),
    ([4], 3, 0),
    ([4], 0, 1), 
    ([4], 5, 0), 
    ([4], 1, 0), 
    ([0, 17], 4, 1),
    ([0, 17], 2, 0),
    ([0, 17], 4, 0), 
    ([0, 17], 4, 1),
    ([0, 17], 1, 0), 
    ([0, 17], 3, 0), 
    ([4], 0, 1),
    ([4], 5, 0), 
    ([4], 0, 0), 
    ([4], 1, 1), 
    ([4], 5, 0),
    ([4], 5, 0),
    ([1, 10, 12], 4, 1),
    ([1, 10, 12], 0, 0),
    ([1, 10, 12], 3, 0),
    ([1, 10, 12], 5, 1),
    ([1, 10, 12], 4, 0),
    ([1, 10, 12], 3, 0),
    ([11, 5, 7], 1, 1), 
    ([11, 5, 7], 3, 0),
    ([11, 5, 7], 0, 0),
    ([11, 5, 7], 4, 1),
    ([11, 5, 7], 1, 0),
    ([11, 5, 7], 4, 0),
    ([4], 5, 1),
    ([4], 0, 0),
    ([4], 0, 0), 
    ([4], 5, 1), 
    ([4], 0, 0),
    ([4], 1, 0), 
    ([11, 5, 7], 4, 1),
    ([11, 5, 7], 1, 0),
    ([11, 5, 7], 4, 0),
    ([11, 5, 7], 4, 1),
    ([11, 5, 7], 4, 0), 
    ([11, 5, 7], 0, 0),
    ([4], 5, 1), 
    ([4], 5, 0), 
    ([4], 0, 0),
    ([4], 5, 1),
    ([4], 5, 0), 
    ([4], 2, 0), 
    ([11, 5, 7], 4, 1),
    ([11, 5, 7], 4, 0),
    ([11, 5, 7], 0, 0),
    ([11, 5, 7], 1, 1),
    ([11, 5, 7], 0, 0),
    ([11, 5, 7], 0, 0), 
    ([1, 10, 12], 5, 1),
    ([1, 10, 12], 3, 0),
    ([1, 10, 12], 5, 0)
]

def read_list():
    return ast.literal_eval(sys.stdin.readline())

def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def make_diag_mask(size, radius):
    """Квадратная матрица размера Size x Size с двумя полосами ширины radius вдоль главной диагонали"""
    idxs = np.arange(size)
    abs_idx_diff = np.abs(np.arange(size)[np.newaxis, :] - np.arange(size)[:, np.newaxis])
    mask = ((abs_idx_diff <= radius // 2) & (abs_idx_diff > 0)).astype(int)
    return mask

def generate_ft_sgns_samples(text, window_size, vocab_size, ns_rate, token2subwords):
    """
    text - list of integer numbers - ids of tokens in text
    window_size - odd integer - width of window
    vocab_size - positive integer - number of tokens in vocabulary
    ns_rate - positive integer - number of negative tokens to sample per one positive sample
    token2subwords - list of lists of int - i-th sublist contains list of identifiers of n-grams for token #i (list of subword units)

    returns list of training samples (CenterSubwords, CtxWord, Label)
    """
    text = np.array(text)
    size = len(text)
    context_words_idxs = make_diag_mask(size, window_size)
    center_words_idx = np.eye(N=size, dtype=int)

    result = []

    for i, word in enumerate(text):
        context = context_words_idxs[i]
        center = center_words_idx[i]

        context_idx = np.where(context != 0)[0]
        center_idx = np.where(center != 0)[0]

        center_word = text[center_idx].tolist()
        context_words = text[context_idx]
        
        center_word.extend(token2subwords[word])

        positive_examples = [tuple(list(j) + [1]) for j in product([center_word], context_words)]    

        # чтобы не совпадало с самим собой
#         all_words = np.arange(vocab_size)
#         center_word_idx = np.argwhere(all_words == center_word)[0][0]
#         all_words = list(all_words)
#         del all_words[center_word_idx]
#         all_words = np.array(all_words)
#         negative_word = np.random.choice(all_words)

        # выбрать одно из двух, либо убрать совпадение с самим собой, либо нет
        negative_words = np.random.randint(vocab_size, size=ns_rate*len(positive_examples))    
        negative_examples = [tuple(list(j) + [0]) for j in product([center_word], negative_words)]

        positive_negative_examples = positive_examples + negative_examples
        result.extend(positive_negative_examples)
    
    return result

result = generate_ft_sgns_samples(*sample_input)

result

[([1, 10, 12], 2, 1),
 ([1, 10, 12], 4, 0),
 ([1, 10, 12], 5, 0),
 ([2, 20, 20], 1, 1),
 ([2, 20, 20], 0, 1),
 ([2, 20, 20], 4, 0),
 ([2, 20, 20], 4, 0),
 ([2, 20, 20], 0, 0),
 ([2, 20, 20], 4, 0),
 ([0, 17], 2, 1),
 ([0, 17], 1, 1),
 ([0, 17], 1, 0),
 ([0, 17], 5, 0),
 ([0, 17], 0, 0),
 ([0, 17], 1, 0),
 ([1, 10, 12], 0, 1),
 ([1, 10, 12], 4, 1),
 ([1, 10, 12], 2, 0),
 ([1, 10, 12], 0, 0),
 ([1, 10, 12], 3, 0),
 ([1, 10, 12], 2, 0),
 ([4], 1, 1),
 ([4], 0, 1),
 ([4], 3, 0),
 ([4], 1, 0),
 ([4], 1, 0),
 ([4], 2, 0),
 ([0, 17], 4, 1),
 ([0, 17], 4, 1),
 ([0, 17], 0, 0),
 ([0, 17], 3, 0),
 ([0, 17], 0, 0),
 ([0, 17], 3, 0),
 ([4], 0, 1),
 ([4], 1, 1),
 ([4], 1, 0),
 ([4], 2, 0),
 ([4], 0, 0),
 ([4], 5, 0),
 ([1, 10, 12], 4, 1),
 ([1, 10, 12], 5, 1),
 ([1, 10, 12], 1, 0),
 ([1, 10, 12], 5, 0),
 ([1, 10, 12], 3, 0),
 ([1, 10, 12], 1, 0),
 ([5, 7, 11], 1, 1),
 ([5, 7, 11], 4, 1),
 ([5, 7, 11], 1, 0),
 ([5, 7, 11], 2, 0),
 ([5, 7, 11], 3, 0),
 ([5, 7, 11], 2, 0),
 ([4], 5, 1),
 ([4], 5, 1),


# 3.4.4

Вы обучаете FastText SkipGram Negative Sampling.

Напишите функцию, обновляющую веса модели при получении одного обучающего примера в формате (CenterSubwords, CtxWord, Label)(CenterSubwords,CtxWord,Label).

При обучении предсказания модели вычисляются по формуле 
$P(CtxWord | CenterSubwords) = \sigma\left(\left(\sum_{w \in CenterSubwords}^{len(CenterSubwords)} \frac {W_{w, :}} {len(CenterSubwords)} \right) \cdot D_{CtxWord,:}\right)$
 
Функция потерь - бинарная кросс-энтропия.

Тесты генерируются случайно.

In [9]:
import sys
import ast
import numpy as np

sample_input = (
    [
        [0.07217140995735816, 0.9807495045952024, 0.5888650678318127, 0.9419020475323008, 0.9698687137771355],
        [0.17481764801167854, 0.9598681333667267, 0.8615416075076997, 0.6649845254089604, 0.14272822189820067],
        [0.695257160390079, 0.6252124583357915, 0.788572884360212, 0.5407620598434707, 0.4742760619803522],
        [0.3720755825170682, 0.8734430653555122, 0.29388553936147677, 0.7833976055802006, 0.11647446813597206],
        [0.4793503066165381, 0.7731679392102295, 0.6466062364447424, 0.5834632727525674, 0.16975097768580916],
        [0.46855676928071344, 0.7440440871653314, 0.5968916205486556, 0.6949993371605877, 0.9995564750677164],
        [0.3995517204225809, 0.30217048674177027, 0.6934836340605662, 0.5025452046745376, 0.43990420866402447],
        [0.6233285824044058, 0.7510765715859197, 0.8764982899024905, 0.42892241183749247, 0.9241569354174014],
        [0.21063022873083803, 0.979366603599722, 0.07879437255385402, 0.7103116511451802, 0.298121842692622],
        [0.7991181799927396, 0.8700912396205017, 0.4936455488806514, 0.9306352063022928, 0.671689987782089], 
        [0.11245515636577097, 0.2591385008756272, 0.38393130144123977, 0.5927928993875077, 0.3343301767582757],
        [0.027340724019638274, 0.15461071231349877, 0.7955192467457007, 0.050624838697975516, 0.26136570172628426],
        [0.7825083895933859, 0.9046538942978853, 0.4559636175207443, 0.733829258685726, 0.022174763292638677],
        [0.6968176063951074, 0.47974647096747125, 0.8885207189970179, 0.016167994434510558, 0.13260182909882334],
        [0.5947903955259933, 0.07459974351651177, 0.11391699485528617, 0.823474357110585, 0.4918622459339238],
        [0.6272760016913231, 0.2711994820963495, 0.24338892914238242, 0.7731707300677505, 0.03720128542002399],
        [0.8640858092433228, 0.027663971153230382, 0.9271422334467209, 0.37457369227183035, 0.17413436429736662],
        [0.4878584763813121, 0.5022845803948351, 0.13899660663745628, 0.8353408935052742, 0.48314336609381436],
        [0.8197910105251979, 0.5371430936015362, 0.12965724315376936, 0.06244349080403733, 0.9558816248633216],
        [0.5929477505385994, 0.36687167726065173, 0.42925321480480627, 0.8435274356179648, 0.8550018469714032], 
        [0.45785273815309, 0.008764229829187009, 0.6840407156586629, 0.04831125277736026, 0.14609911971743395],
        [0.1579479219010974, 0.1298470924838635, 0.8283362978065627, 0.9140741421274726, 0.7516395431217443],
        [0.01139316661353773, 0.6980229640742956, 0.45528806869472405, 0.7653990849713008, 0.24848012670857944],
        [0.8750941097872984, 0.6964598870452183, 0.6675389863133752, 0.391939718013135, 0.30592620271209714], 
        [0.024161748164975072, 0.6512328549928654, 0.27784751504029503, 0.32588414662648524, 0.4073676483413957],
        [0.7372935688667617, 0.9743689028772393, 0.26179932035274445, 0.3556999822154028, 0.8234406534181563],
        [0.9358431512408416, 0.0030942521035778325, 0.7052198210371732, 0.3494249594704901, 0.06494462197366668],
        [0.027642224051125597, 0.45820907093457997, 0.6172763215932299, 0.03520578036716404, 0.05004091043245007]
    ],
    [
        [0.3619192935809462, 0.7910582560833153, 0.173840770588212, 0.8486217599360419, 0.09895998679198104],
        [0.9524670374363299, 0.577316446205222, 0.3348594666828074, 0.7987547183235284, 0.710457681490417],
        [0.8400820704952479, 0.9414962586451427, 0.08399082278691339, 0.425927381574433, 0.6304514720560764],
        [0.5331686510681622, 0.2751366715811131, 0.8329999135745643, 0.2770290564458684, 0.020564166091874392],
        [0.9852792048968001, 0.922320208232837, 0.7297936992308128, 0.20212997935663524, 0.5277458149323955],
        [0.43383566311415755, 0.14151987203148808, 0.3267585826852797, 0.8796734627573763, 0.14253685112772174],
        [0.24559727482999572, 0.3015598034026842, 0.12351719983998721, 0.6141130319406622, 0.9210871618079258],
        [0.21915908704207665, 0.9809645232509783, 0.8685879466971278, 0.9956335594634693, 0.0441562419906687],
        [0.24988758739587902, 0.42298807118368675, 0.01922872769211703, 0.02806386746602596, 0.2821901214584819],
        [0.43997555452635384, 0.5078839449569567, 0.812607950040521, 0.9998014106280365, 0.1559607489614684],
        [0.9092151190046189, 0.5930002929595868, 0.315159378929991, 0.4052299042409616, 0.984475831988958], 
        [0.7836990450026143, 0.002466529016497798, 0.8465916260137056, 0.7227126698344118, 0.5087557482398855],
        [0.4125921074144525, 0.5582795115000383, 0.889307828978137, 0.928416977596577, 0.8437462138575066], 
        [0.11810981794872477, 0.07787452990697508, 0.3907338451314212, 0.6841828899516664, 0.4547615738832046],
        [0.4977766315279062, 0.09878866849137813, 0.0622140049250518, 0.9008881823827194, 0.3694055807903669],
        [0.12415427540834822, 0.01064247175537103, 0.1439469061372417, 0.43996173718103593, 0.3846553735294024],
        [0.36544315427420426, 0.6651402425072226, 0.3837201693785094, 0.54713466624535, 0.6925194086063208], 
        [0.8217730539154436, 0.7380601103419114, 0.4790971996703556, 0.935248458815274, 0.6385239169547122], 
        [0.4884363834477089, 0.783319748626155, 0.018212966919229467, 0.03662832627793777, 0.03532160993715294], 
        [0.6820505211290306, 0.25769913167047753, 0.9677388106523852, 0.4471332422618759, 0.7731319006564568], 
        [0.3695513424667971, 0.5118113495291988, 0.1721439269100805, 0.09451631327113852, 0.8369170475041434], 
        [0.7918542552021289, 0.0245240901264403, 0.6658133706965796, 0.9740885323982209, 0.02660284500887522], 
        [0.5604137104962275, 0.5643917632639455, 0.6756476068355826, 0.9466913679034125, 0.21062462975598062], 
        [0.7306868573812846, 0.7573083135261555, 0.9450278665003865, 0.9649869335038909, 0.1262321882978371], 
        [0.6830284536315845, 0.7383035166437748, 0.7985226892860073, 0.005247820534787007, 0.6886083391552933],
        [0.6905561126225058, 0.3220803445510755, 0.8885006766287556, 0.32709316933290455, 0.9126547743770385],
        [0.26866358146648694, 0.9355232286537734, 0.5254946965960933, 0.6487428023364232, 0.9405298594379049], 
        [0.33881123962516546, 0.6820622877451537, 0.3053828831926755, 0.9229486901650673, 0.5450270097149575]
    ],
    [3],
    1,
    1,
    0.8562235244377375
)

sample_output = (
    [
        [0.07217140995735816, 0.9807495045952024, 0.5888650678318127, 0.9419020475323008, 0.9698687137771355],
        [0.17481764801167854, 0.9598681333667267, 0.8615416075076997, 0.6649845254089604, 0.14272822189820067],
        [0.695257160390079, 0.6252124583357915, 0.788572884360212, 0.5407620598434707, 0.4742760619803522], 
        [0.5017594511598129, 0.9520480219915879, 0.3394785828834429, 0.8921526573535964, 0.21320737019064417],
        [0.4793503066165381, 0.7731679392102295, 0.6466062364447424, 0.5834632727525674, 0.16975097768580916],
        [0.46855676928071344, 0.7440440871653314, 0.5968916205486556, 0.6949993371605877, 0.9995564750677164], 
        [0.3995517204225809, 0.30217048674177027, 0.6934836340605662, 0.5025452046745376, 0.43990420866402447],
        [0.6233285824044058, 0.7510765715859197, 0.8764982899024905, 0.42892241183749247, 0.9241569354174014], 
        [0.21063022873083803, 0.979366603599722, 0.07879437255385402, 0.7103116511451802, 0.298121842692622], 
        [0.7991181799927396, 0.8700912396205017, 0.4936455488806514, 0.9306352063022928, 0.671689987782089],
        [0.11245515636577097, 0.2591385008756272, 0.38393130144123977, 0.5927928993875077, 0.3343301767582757],
        [0.027340724019638274, 0.15461071231349877, 0.7955192467457007, 0.050624838697975516, 0.26136570172628426],
        [0.7825083895933859, 0.9046538942978853, 0.4559636175207443, 0.733829258685726, 0.022174763292638677],
        [0.6968176063951074, 0.47974647096747125, 0.8885207189970179, 0.016167994434510558, 0.13260182909882334],
        [0.5947903955259933, 0.07459974351651177, 0.11391699485528617, 0.823474357110585, 0.4918622459339238],
        [0.6272760016913231, 0.2711994820963495, 0.24338892914238242, 0.7731707300677505, 0.03720128542002399],
        [0.8640858092433228, 0.027663971153230382, 0.9271422334467209, 0.37457369227183035, 0.17413436429736662],
        [0.4878584763813121, 0.5022845803948351, 0.13899660663745628, 0.8353408935052742, 0.48314336609381436], 
        [0.8197910105251979, 0.5371430936015362, 0.12965724315376936, 0.06244349080403733, 0.9558816248633216], 
        [0.5929477505385994, 0.36687167726065173, 0.42925321480480627, 0.8435274356179648, 0.8550018469714032],
        [0.45785273815309, 0.008764229829187009, 0.6840407156586629, 0.04831125277736026, 0.14609911971743395], 
        [0.1579479219010974, 0.1298470924838635, 0.8283362978065627, 0.9140741421274726, 0.7516395431217443], 
        [0.01139316661353773, 0.6980229640742956, 0.45528806869472405, 0.7653990849713008, 0.24848012670857944],
        [0.8750941097872984, 0.6964598870452183, 0.6675389863133752, 0.391939718013135, 0.30592620271209714],
        [0.024161748164975072, 0.6512328549928654, 0.27784751504029503, 0.32588414662648524, 0.4073676483413957],
        [0.7372935688667617, 0.9743689028772393, 0.26179932035274445, 0.3556999822154028, 0.8234406534181563],
        [0.9358431512408416, 0.0030942521035778325, 0.7052198210371732, 0.3494249594704901, 0.06494462197366668], 
        [0.027642224051125597, 0.45820907093457997, 0.6172763215932299, 0.03520578036716404, 0.05004091043245007]
    ],
    [
        [0.3619192935809462, 0.7910582560833153, 0.173840770588212, 0.8486217599360419, 0.09895998679198104],
        [1.0031272693097524, 0.6962407462622227, 0.37487367419295825, 0.9054188108159625, 0.726316350643562],
        [0.8400820704952479, 0.9414962586451427, 0.08399082278691339, 0.425927381574433, 0.6304514720560764], 
        [0.5331686510681622, 0.2751366715811131, 0.8329999135745643, 0.2770290564458684, 0.020564166091874392],
        [0.9852792048968001, 0.922320208232837, 0.7297936992308128, 0.20212997935663524, 0.5277458149323955], 
        [0.43383566311415755, 0.14151987203148808, 0.3267585826852797, 0.8796734627573763, 0.14253685112772174], 
        [0.24559727482999572, 0.3015598034026842, 0.12351719983998721, 0.6141130319406622, 0.9210871618079258], 
        [0.21915908704207665, 0.9809645232509783, 0.8685879466971278, 0.9956335594634693, 0.0441562419906687],
        [0.24988758739587902, 0.42298807118368675, 0.01922872769211703, 0.02806386746602596, 0.2821901214584819],
        [0.43997555452635384, 0.5078839449569567, 0.812607950040521, 0.9998014106280365, 0.1559607489614684],
        [0.9092151190046189, 0.5930002929595868, 0.315159378929991, 0.4052299042409616, 0.984475831988958],
        [0.7836990450026143, 0.002466529016497798, 0.8465916260137056, 0.7227126698344118, 0.5087557482398855],
        [0.4125921074144525, 0.5582795115000383, 0.889307828978137, 0.928416977596577, 0.8437462138575066],
        [0.11810981794872477, 0.07787452990697508, 0.3907338451314212, 0.6841828899516664, 0.4547615738832046],
        [0.4977766315279062, 0.09878866849137813, 0.0622140049250518, 0.9008881823827194, 0.3694055807903669], 
        [0.12415427540834822, 0.01064247175537103, 0.1439469061372417, 0.43996173718103593, 0.3846553735294024],
        [0.36544315427420426, 0.6651402425072226, 0.3837201693785094, 0.54713466624535, 0.6925194086063208],
        [0.8217730539154436, 0.7380601103419114, 0.4790971996703556, 0.935248458815274, 0.6385239169547122],
        [0.4884363834477089, 0.783319748626155, 0.018212966919229467, 0.03662832627793777, 0.03532160993715294],
        [0.6820505211290306, 0.25769913167047753, 0.9677388106523852, 0.4471332422618759, 0.7731319006564568],
        [0.3695513424667971, 0.5118113495291988, 0.1721439269100805, 0.09451631327113852, 0.8369170475041434],
        [0.7918542552021289, 0.0245240901264403, 0.6658133706965796, 0.9740885323982209, 0.02660284500887522],
        [0.5604137104962275, 0.5643917632639455, 0.6756476068355826, 0.9466913679034125, 0.21062462975598062],
        [0.7306868573812846, 0.7573083135261555, 0.9450278665003865, 0.9649869335038909, 0.1262321882978371], 
        [0.6830284536315845, 0.7383035166437748, 0.7985226892860073, 0.005247820534787007, 0.6886083391552933], 
        [0.6905561126225058, 0.3220803445510755, 0.8885006766287556, 0.32709316933290455, 0.9126547743770385],
        [0.26866358146648694, 0.9355232286537734, 0.5254946965960933, 0.6487428023364232, 0.9405298594379049],
        [0.33881123962516546, 0.6820622877451537, 0.3053828831926755, 0.9229486901650673, 0.5450270097149575]
    ]
)

def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    s = 1 / (1 + np.exp(-z))
    return s * (1 - s)

def log_loss(y_true, a_pred):
    '''
    Compute log loss, a_pred - vector of size n_objects
    '''
    return np.mean(-y_true * np.log(a_pred) - (1 - y_true) * np.log(1 - a_pred))

def log_loss_derivative(y_true, a_pred):
    '''
    Compute detivative of log loss
    '''
    return (-y_true / a_pred + (1 - y_true) / (1 - a_pred)) / len(y_true)

def update_ft_weights(center_embeddings, context_embeddings, center_subwords, context_word, label, learning_rate):
    """
    center_embeddings - VocabSize x EmbSize
    context_embeddings - VocabSize x EmbSize
    center_subwords - list of ints - list of identifiers of n-grams contained in center word
    context_word - int - identifier of context word
    label - 1 if context_word is real, 0 if it is negative
    learning_rate - float > 0 - size of gradient step
    """
    # Переведем в numpy для удобства вычислений
    center_embeddings = np.asarray(center_embeddings)
    context_embeddings = np.asarray(context_embeddings)
    
    # Выберем центральное и контекстное слово
    center_embedding = center_embeddings[center_subwords]
    context_embedding = context_embeddings[[context_word]]
    
    # Считаем "оценку" сходства и ее вероятность по формуле
    # 𝑃(𝐶𝑡𝑥𝑊𝑜𝑟𝑑|𝐶𝑒𝑛𝑡𝑒𝑟𝑆𝑢𝑏𝑤𝑜𝑟𝑑𝑠)=𝜎((∑𝑙𝑒𝑛(𝐶𝑒𝑛𝑡𝑒𝑟𝑆𝑢𝑏𝑤𝑜𝑟𝑑𝑠)𝑤∈𝐶𝑒𝑛𝑡𝑒𝑟𝑆𝑢𝑏𝑤𝑜𝑟𝑑𝑠𝑊𝑤,:𝑙𝑒𝑛(𝐶𝑒𝑛𝑡𝑒𝑟𝑆𝑢𝑏𝑤𝑜𝑟𝑑𝑠))⋅𝐷𝐶𝑡𝑥𝑊𝑜𝑟𝑑,:)
    score = np.mean(center_embedding, axis=0, keepdims=True).dot(context_embedding.T)
    prob = sigmoid(score)
    print(f'LOSS: {log_loss(label, prob)}')
    
    # Считаем производные для правила цепочки: производная лосса * производную сигмоиды
    # * производную 𝑊𝐶𝑒𝑛𝑡𝑒𝑟𝑊𝑜𝑟𝑑,:⋅𝐷𝐶𝑡𝑥𝑊𝑜𝑟𝑑,: по 𝑊𝐶𝑒𝑛𝑡𝑒𝑟𝑊𝑜𝑟𝑑 и 𝐷𝐶𝑡𝑥𝑊𝑜𝑟𝑑
    log_loss_deriv = log_loss_derivative(np.array([label]), np.array(prob))
    sigmoid_deriv = sigmoid_derivative(score)
    
    # Считаем производные по цетральному слову и контекстному для обновления весов
    center_grad = log_loss_deriv * sigmoid_deriv * context_embedding / len(center_subwords)
    context_grad = log_loss_deriv * sigmoid_deriv * np.mean(center_embedding, axis=0, keepdims=True)
    print(f'Center word grad: {center_grad}')
    print(f'Context word grad: {context_grad}')
    
    center_embeddings[center_subwords] -= learning_rate * center_grad
    context_embeddings[[context_word]] -= learning_rate * context_grad
    
    return center_embeddings, context_embeddings


result = update_ft_weights(*sample_input)

print(np.allclose(result[0], np.array(sample_output[0])))
print(np.allclose(result[1], np.array(sample_output[1])))

LOSS: 0.17318613700389962
Center word grad: [[-0.1514603  -0.09180425 -0.053249   -0.12701713 -0.11297623]]
Context word grad: [[-0.05916706 -0.13889399 -0.04673337 -0.12457505 -0.01852165]]
True
True


## 3.4.5

Вы собираетесь обучать GloVe и строите матрицу совместной встречаемости токенов, то есть матрицу, в ij ячейке которой стоит количество документов, в которых употреблялось и слово i и слово j. На главной диагонали должны стоять нули (то есть не нужно считать количество совместных употреблений слова с самим собой).

Напишите функцию, которая получает на вход список документов и строит такую матрицу. Документы уже токенизированы и токены представлены их числовыми идентификаторами.

Для хранения матрицы используйте `scipy.sparse.dok_matrix`.

Тесты генерируются случайно.

In [10]:
import sys
import ast
import numpy as np
import scipy.sparse
from itertools import combinations

sample_input = (
    [
        [0, 2, 2, 2, 0, 0],
        [1, 1, 2, 1, 1],
        [2, 2, 1, 1]
    ],
    3
)

sample_output = [[0.0, 0.0, 1.0], [0.0, 0.0, 2.0], [1.0, 2.0, 0.0]]

def generate_coocurrence_matrix(texts, vocab_size):
    """
    texts - list of lists of ints - i-th sublist contains identifiers of tokens in i-th document
    vocab_size - int - size of vocabulary
    returns scipy.sparse.dok_matrix
    """
    
    vocab = np.arange(vocab_size)
    coocurence = scipy.sparse.dok_matrix((vocab_size, vocab_size))
    combs = list(combinations(vocab, 2))
    
    for doc in texts:
        for i, j in combs:
            if i in doc and j in doc:
                coocurence[i, j] += 1
                coocurence[j, i] += 1
    return coocurence


result = generate_coocurrence_matrix(*sample_input)

result.todense()

matrix([[0., 0., 1.],
        [0., 0., 2.],
        [1., 2., 0.]])

## 3.4.6

Вы обучаете GloVe. Матрица совместной встречаемости $X \in \mathbb{R}^{|Vocab| \times |Vocab|}$ построена.

Напишите функцию, которая обновляет параметры модели градиентным спуском (делает один градиентный шаг). Обратите внимание, что Вы должны изменить значения массивов w и d (inplace), а не создать новые массивы с новыми значениями.

Функция потерь: $GloVeLoss(W, D, X) = \sum_{ij} f(x_{ij})(\ln(1+x_{ij}) - W_{i,:} \cdot D_{:,j})^2 \rightarrow \min_{W,D}$

Возможно, будет полезно пересмотреть [фрагмент лекции про GloVe](https://stepik.org/lesson/225313/step/5?unit=198056).

Тесты генерируются случайно.

In [11]:
import sys
import ast
import numpy as np

sample_input = (
    [
        [72, 67, 24, 81, 52, 43, 49, 12, 84, 77, 22, 66, 66, 0, 59, 4, 71, 78, 37, 69, 39, 63, 68, 36, 97, 17],
        [72, 38, 34, 43, 11, 46, 91, 96, 43, 4, 80, 77, 19, 18, 39, 8, 43, 58, 59, 43, 40, 55, 14, 96, 90, 43],
        [83, 82, 22, 93, 5, 17, 68, 30, 2, 67, 7, 8, 34, 2, 88, 66, 31, 52, 96, 13, 9, 83, 3, 9, 91, 15],
        [73, 25, 40, 82, 42, 30, 79, 77, 15, 76, 65, 6, 7, 44, 98, 88, 65, 74, 33, 48, 61, 54, 64, 28, 49, 38],
        [47, 41, 2, 54, 5, 13, 36, 0, 97, 15, 80, 90, 38, 27, 24, 31, 32, 20, 77, 20, 8, 11, 24, 19, 77, 23],
        [55, 26, 42, 5, 98, 87, 36, 1, 11, 19, 57, 68, 92, 49, 98, 9, 98, 24, 0, 13, 14, 90, 10, 51, 30, 30],
        [98, 12, 2, 66, 27, 12, 12, 60, 46, 71, 89, 82, 75, 49, 5, 77, 52, 96, 29, 32, 51, 71, 45, 16, 74, 12],
        [18, 92, 95, 62, 51, 65, 1, 49, 51, 62, 16, 64, 97, 48, 78, 14, 90, 50, 43, 49, 59, 11, 75, 50, 60, 2],
        [47, 45, 88, 78, 93, 30, 79, 20, 69, 68, 6, 76, 41, 3, 57, 98, 62, 6, 65, 53, 7, 9, 76, 96, 19, 88], 
        [45, 73, 39, 70, 21, 62, 82, 13, 14, 72, 8, 23, 99, 49, 33, 80, 21, 67, 37, 31, 38, 48, 40, 61, 61, 67],
        [61, 86, 91, 61, 13, 88, 79, 56, 78, 87, 91, 94, 37, 14, 15, 44, 91, 3, 6, 23, 15, 85, 18, 58, 11, 4],
        [50, 28, 55, 44, 21, 62, 98, 64, 85, 84, 4, 31, 59, 16, 51, 11, 37, 44, 6, 60, 47, 54, 70, 29, 32, 74],
        [39, 6, 17, 54, 15, 71, 24, 94, 5, 16, 15, 74, 43, 98, 75, 10, 79, 78, 99, 47, 99, 4, 22, 90, 12, 19], 
        [74, 51, 67, 72, 21, 9, 57, 50, 0, 43, 80, 91, 58, 46, 92, 98, 11, 4, 36, 31, 90, 90, 91, 52, 68, 63],
        [95, 76, 24, 52, 3, 71, 19, 75, 34, 92, 83, 15, 77, 12, 96, 58, 63, 68, 75, 9, 28, 44, 30, 94, 67, 49],
        [22, 93, 33, 77, 2, 9, 3, 3, 47, 56, 84, 70, 15, 81, 16, 49, 20, 95, 18, 22, 98, 3, 77, 27, 1, 13],
        [45, 63, 34, 0, 75, 45, 30, 23, 7, 7, 80, 62, 34, 11, 41, 16, 45, 6, 11, 21, 18, 55, 7, 24, 18, 70],
        [13, 7, 21, 85, 29, 53, 56, 83, 63, 89, 18, 67, 93, 73, 37, 3, 55, 65, 16, 72, 6, 80, 0, 39, 51, 24],
        [72, 23, 9, 56, 60, 88, 69, 6, 8, 92, 3, 44, 29, 5, 58, 58, 55, 24, 48, 57, 28, 69, 64, 72, 58, 98],
        [20, 56, 52, 74, 27, 95, 85, 20, 7, 52, 8, 93, 76, 53, 62, 54, 34, 25, 89, 38, 85, 29, 38, 18, 1, 28],
        [30, 97, 74, 11, 36, 92, 55, 74, 34, 29, 12, 40, 61, 69, 54, 72, 14, 64, 73, 75, 75, 4, 37, 47, 17, 29], 
        [7, 18, 25, 6, 51, 63, 63, 53, 38, 96, 19, 56, 36, 35, 75, 99, 32, 28, 68, 14, 55, 9, 3, 19, 9, 59],
        [0, 98, 57, 98, 13, 25, 55, 56, 58, 37, 30, 90, 51, 71, 10, 36, 58, 94, 32, 80, 95, 44, 40, 82, 99, 6], 
        [74, 28, 93, 37, 81, 54, 92, 89, 52, 96, 93, 8, 65, 82, 7, 14, 75, 0, 45, 59, 15, 17, 85, 87, 10, 52],
        [10, 74, 13, 23, 56, 25, 66, 59, 86, 39, 47, 72, 92, 28, 23, 75, 23, 18, 5, 20, 36, 52, 42, 56, 20, 7], 
        [32, 37, 58, 20, 3, 33, 76, 92, 36, 73, 90, 53, 82, 78, 6, 66, 11, 33, 64, 68, 51, 76, 94, 94, 74, 88]
    ],
    [
        [0.7236403458959406, 0.0956019387576047, 0.0025299248050427714, 0.8219024304497274, 0.43253754513562515, 0.8013795226500925],
        [0.1645225418615962, 0.17254764305062675, 0.915834884927677, 0.15659274788174238, 0.4408801726853846, 0.6712507398638423],
        [0.7220314060070252, 0.1109087497279424, 0.8673890374761482, 0.6019681601593759, 0.21136092547712715, 0.46410460250177055],
        [0.2051472970020488, 0.7021578939163269, 0.4920315519905448, 0.8786530949689468, 0.8406582658875078, 0.7656322995670249], 
        [0.5314722945128192, 0.20582039242966288, 0.6649783801689887, 0.9122470167268962, 0.06046820688028054, 0.7640361944809368],
        [0.8531299103217095, 0.8837919293919477, 0.5584731093192602, 0.5488851769744959, 0.5426259488733682, 0.8101919492091457], 
        [0.014691936047236509, 0.8299297933323541, 0.04420642840864686, 0.19514486051010316, 0.5605834763387445, 0.021425480951998255],
        [0.6251450063221531, 0.916013278510962, 0.9266733043623226, 0.4314909906070713, 0.5861250222822415, 0.6933275681854775], 
        [0.613436662402963, 0.25971014970117345, 0.8516017571376222, 0.3946078968050868, 0.5030576607821642, 0.4947379037657953], 
        [0.1831704150579163, 0.7027400367924451, 0.9687380255252486, 0.05161874874595729, 0.5662001008903554, 0.1163342848387866],
        [0.7871817922619593, 0.2744881375377821, 0.47927673745333677, 0.29916960674176196, 0.36165825794500894, 0.4473132902029585],
        [0.9460136327515588, 0.9784510345542305, 0.7292583652396575, 0.9967710630754123, 0.5222338378761943, 0.15774056366446398], 
        [0.5663154377790205, 0.31992559317458047, 0.895903341426127, 0.10800834113175917, 0.7025174488794499, 0.09983260287294515],
        [0.2870859802344229, 0.6124244361792336, 0.03043370710190285, 0.4177754705816856, 0.41076530192454186, 0.059229404317664214],
        [0.453421549409623, 0.10006035499361832, 0.4729640823042591, 0.4187735846604017, 0.19252902582118436, 0.4571615927038022],
        [0.4717366823003555, 0.311470963714212, 0.7563429074261462, 0.9450429903711869, 0.23851560864324461, 0.4264206092799121],
        [0.14209483434392234, 0.9545183136517666, 0.02853067102355411, 0.8397788414889452, 0.28747164060068653, 0.5890799959267197],
        [0.7137144457967627, 0.7108041311984524, 0.3391605131543025, 0.18466650700703768, 0.07037926283668172, 0.1691030355977058],
        [0.4181167385409663, 0.5733773938988352, 0.9308794863064511, 0.955104551017489, 0.7472618752255964, 0.9106883383537705], 
        [0.29209827546854006, 0.7950653331872178, 0.9314779081831699, 0.2137419943082265, 0.9590688802321072, 0.21779623076769017],
        [0.6414528631722118, 0.7772400748403205, 0.7240597746441493, 0.4846785371953165, 0.20903895145878393, 0.9928711008461597],
        [0.4987552039133927, 0.966261456826001, 0.6392910461562884, 0.3891694028095307, 0.14376415691424704, 0.5654942409405452],
        [0.39062876410463865, 0.4372793328535669, 0.9066881332880398, 0.928194141998039, 0.26891611788606773, 0.970014111003586], 
        [0.05753018657343756, 0.5987554892139141, 0.6695393400712614, 0.4342378657370556, 0.5068004463455815, 0.28913437767829675],
        [0.31284712702847906, 0.6696586256781413, 0.6349611781499843, 0.11008282689008553, 0.9000387199581723, 0.5893732652223279],
        [0.38771861901614457, 0.9275236976874062, 0.1507893346167909, 0.2649576462980838, 0.8917999241041804, 0.7060665522096253]
    ],
    [
        [0.7146175764456325, 0.31161087332596693, 0.799868898982844, 0.3303984762074823, 0.15755367025489198, 0.8822561515814714],
        [0.48324415449065805, 0.32294633607735035, 0.273076894762348, 0.46575965932905583, 0.35173647464295466, 0.0698782343365999],
        [0.05951092454514029, 0.9631544906381114, 0.14919875559361273, 0.9071033838543416, 0.9235221236014998, 0.15343960980130578],
        [0.37667471994346735, 0.3832592710693109, 0.1372971042292026, 0.5063394603470396, 0.3657347277059969, 0.21520394748123772],
        [0.5589413502705171, 0.9228726685280682, 0.9028006349689756, 0.7902921185261006, 0.09337560160131464, 0.8806823905125992], 
        [0.19078196327854235, 0.9862705503057667, 0.00800242331367751, 0.7036641885324555, 0.7452071471082209, 0.85314563397203], 
        [0.42059808696733225, 0.3678976649279547, 0.15153142787888962, 0.9831212856723789, 0.7218055186807681, 0.8943329971799489],
        [0.058672278269596756, 0.6364681756816949, 0.24610924719747507, 0.8429515887557353, 0.23639035927773622, 0.9193123017124043],
        [0.3667295853360063, 0.46010540148263646, 0.818107188288508, 0.027140526241385965, 0.4420026102807323, 0.3050634480740779],
        [0.9602073143407148, 0.5408373879572825, 0.4027285042008486, 0.854769594232319, 0.8977332204421882, 0.7804511190784789], 
        [0.9554030213710992, 0.6286064807931032, 0.7899715293283952, 0.20778805629585584, 0.34452317136784105, 0.8373278109724016],
        [0.9511367017094053, 0.8108673965379353, 0.5917802839407773, 0.08638924272725734, 0.5614389008614823, 0.10285577516634681], 
        [0.15293610366355237, 0.4726630546010566, 0.7151593451811216, 0.6787398883364194, 0.05726564336395312, 0.11175850750236216],
        [0.5284613368117816, 0.2171952870224766, 0.14730305464381344, 0.16327154211081985, 0.22473713798444594, 0.8780618686814565],
        [0.6229749344457463, 0.2450307938022901, 0.856716114606889, 0.5130970556519491, 0.09638792050417233, 0.5580480219996736],
        [0.02557257524793899, 0.16614696997999867, 0.9057474930205891, 0.9639638373151679, 0.8305505098688646, 0.13212730388642224],
        [0.9945224728362285, 0.601015567237635, 0.627777689771871, 0.062014306890884385, 0.5482657713187832, 0.050645865034282034],
        [0.222530564647055, 0.16270913407631815, 0.3463743065572499, 0.3642479732760492, 0.6787809827842912, 0.6698646733332234], 
        [0.0514618465561959, 0.09146560753484756, 0.5663782403169225, 0.09809277695721963, 0.7435268283749681, 0.6941669527997202], 
        [0.7220783710745816, 0.242189075941737, 0.19197963437514165, 0.2789768605860322, 0.1257100212184865, 0.25803379668907667],
        [0.5231678066470311, 0.4611093289035184, 0.8420569136872692, 0.9566490894261072, 0.07691192438283945, 0.37613366065780873], 
        [0.010481514678729265, 0.5145103851754453, 0.9425491945781952, 0.24440293940943314, 0.2766636476384883, 0.9944680222564074],
        [0.7081598239606982, 0.3291415847107684, 0.7986116830970068, 0.32005951294163504, 0.988878016430301, 0.16702718654180948],
        [0.3281899603978248, 0.8371583970360936, 0.914781121298489, 0.9898376984561366, 0.2605393835200668, 0.7046307961318979],
        [0.6669241697435273, 0.7506837943872975, 0.3223310011040579, 0.8024412673323509, 0.47139557621217376, 0.34991596973647043],
        [0.6981065171211724, 0.7907802796637423, 0.05700463849852977, 0.29301210116680565, 0.3246921756526622, 0.9147896908728982]
    ],
    0.5878728651551414,
    97,
    0.8066056621137515
)

sample_output = (
    [
        [37.894949622265095, 35.32608453541813, 39.58755103784916, 33.56742861491748, 31.889707572091282, 33.00715086688418],
        [33.84381119427707, 38.036945787753176, 34.69008748288071, 36.55796979484229, 30.50706391340045, 40.418661343576375],
        [22.292630735670524, 22.26123383895233, 27.281358895160945, 29.762116352667036, 26.093786102891865, 28.873616599806525],
        [28.627244763428482, 25.010240783610097, 30.24910190951392, 29.94251520200709, 25.793350857840288, 31.500830971429988],
        [24.86914949950388, 21.18902354047288, 25.17742133333728, 16.734270195198985, 21.32882188455089, 20.133684294288226], 
        [20.607435095197772, 23.38444651793354, 24.170604089788068, 19.190985999049406, 14.732626697936645, 22.085786623565003],
        [41.83248643873107, 39.23972477624278, 48.25206831443954, 39.18447489715208, 36.223555643455576, 43.838987581052415], 
        [25.957662536809675, 27.78414465308494, 26.831852326280252, 27.670481277635595, 24.681666254723726, 21.151581083066553],
        [32.851892878431265, 36.440775336903165, 35.75352506946652, 37.30323390355503, 34.53270452844875, 31.563665533166038],
        [30.835038636245486, 34.52475728758245, 33.16493620256741, 41.27828441760409, 33.625782386520584, 37.29511586329691], 
        [33.83703170479853, 40.43002001773247, 36.184443329518004, 37.12624122167327, 35.08771179980609, 35.139255640003434], 
        [21.254977902045137, 23.060454293882493, 21.303095809926, 23.591298842309634, 20.611990194926875, 24.660270597555062], 
        [29.841626803079436, 31.839284837338443, 32.251032295284844, 33.58647928947483, 26.771111486581926, 37.72191514982901],
        [44.55346815719802, 45.05766658559051, 52.471710160526506, 49.79668060973384, 39.22443377310308, 47.134158002890395], 
        [39.17357712318688, 41.7702595729068, 45.89022396002535, 45.0492861943433, 37.91715762872248, 46.34889394407784],
        [27.66000266779389, 21.58602345742282, 26.20679184375472, 22.50135429223267, 24.80834623255724, 21.774444708488712],
        [24.235631356398315, 25.330637975611467, 24.35882534462737, 19.614644539141796, 16.096340205031563, 23.969425116432753],
        [32.714784659285534, 37.00623479448081, 32.80393996121548, 35.850620589216966, 29.775563785011812, 40.1682995737016], 
        [22.473439079254767, 21.63905155108375, 21.309990260094732, 21.384623495336577, 20.03459479396905, 23.052219798176157],
        [25.833857096031426, 28.006935843177743, 27.529323359718862, 31.562166519759444, 27.60438145889944, 28.783954098290046],
        [21.724755945837586, 25.46457829943065, 25.453384408355255, 30.825724432379566, 26.016894561298137, 26.62071479383954],
        [21.81725003511163, 24.05930480422645, 25.802129103735567, 28.29934558071534, 24.882668372989844, 27.55650443868176],
        [29.608735571248964, 29.25708252569204, 28.139612290451367, 30.385015579072235, 27.487966113975027, 24.962848111956525],
        [39.403771170848984, 41.917746619056196, 37.84557861173111, 40.69821075997771, 35.62873141884051, 45.079155106826896],
        [23.04120185595438, 26.786780888021532, 31.2639541627983, 29.00755536709279, 22.203982913169703, 24.495657931209138],
        [32.403129266441965, 35.840017821814904, 37.17203998680061, 38.03281061366912, 32.78714804739385, 39.28183659620253]
    ],
    [
        [25.737584343831532, 31.976583863756073, 31.991796072794237, 30.397770296625676, 29.155071762578395, 28.039906267294363],
        [39.18090234664077, 43.45477045037197, 51.786860206912635, 43.09517606909757, 36.467098828996626, 45.87756004198227],
        [25.861638022497715, 30.92652100750488, 33.60725677532385, 25.184249360983333, 25.33461010279592, 26.93195963760828],
        [40.44255724281372, 42.96387247113741, 54.334811744166686, 43.93191831350745, 40.27560778331666, 40.23238565434164], 
        [12.60992409857056, 19.726007962924154, 16.071644794959607, 14.161701223195637, 15.198176415121148, 15.276991028259971], 
        [29.04588467665483, 33.80505200039617, 38.04855856282448, 24.880963308310946, 27.92967131290679, 28.76950184954046], 
        [30.237437485971547, 38.66537897520929, 42.70573743205196, 29.776069477137508, 33.905932759342974, 31.963285050639865],
        [28.028933057301142, 38.08684708617139, 37.544542228274906, 25.321920916303146, 31.533875714831566, 29.298469894937988],
        [30.71424884632851, 30.95498799285244, 34.15309776359906, 31.217888494294368, 24.993625601516545, 31.928661444788062], 
        [26.146184710215547, 32.46478475669627, 31.62669482728407, 25.64499465851579, 26.152486540238574, 24.145438152077382], 
        [18.513444310864926, 28.714219341908812, 22.747896951556424, 25.100699161335342, 23.409036420228585, 24.514405065630598],
        [34.97755034206531, 41.735458041689185, 42.23493985650944, 36.009265517304414, 34.352119472664846, 37.644071472340144],
        [40.092702168388236, 53.62113178600554, 48.84356458863764, 36.39043705817393, 42.28637968655391, 40.280349955856444], 
        [27.09610113742262, 41.26880485353593, 39.346977655994735, 27.591514389672355, 31.995616644580924, 28.87474376777405], 
        [32.46320730114356, 36.48181104762826, 38.714631286168924, 32.33019087297395, 30.27052570417721, 32.240695698900474], 
        [24.62678794161355, 36.21315596132247, 34.91748692038032, 27.31267275243153, 29.942035228767967, 28.65251738118413],
        [31.908551549750985, 35.150861854275824, 38.81804750019257, 33.07677850315476, 30.681610274970396, 34.06666656257596],
        [30.48008982537277, 35.58865452115646, 40.57350768906002, 33.041098811455335, 29.65449752756178, 32.70055131481196],
        [30.57739739495991, 34.19410957043535, 44.6772377886788, 31.000103986208202, 31.17905081422838, 34.40131935305819],
        [32.07230397160679, 40.95180323381044, 41.63975849206306, 34.856807118616764, 34.237546314068695, 36.52332793161486],
        [22.165179600850514, 32.94179603315074, 32.363977866166756, 25.282871906957595, 28.351785486262752, 24.632822374929177], 
        [28.16174839655334, 35.544107679226634, 28.118950582145626, 28.3788129267007, 28.77498009421306, 28.156490872467124],
        [22.230950864818148, 30.475956010093874, 28.654962541126377, 28.12410255465114, 28.61378026798125, 25.497683394160806],
        [22.53316383674741, 26.507404577732675, 32.26317495605627, 20.718985703375136, 26.011612018642545, 25.31060496066265],
        [24.59110367631163, 27.543472573592013, 30.621636332862288, 28.235921081852787, 24.45902884286537, 29.64689202372981], 
        [21.024604742056106, 30.902045272493908, 29.293039097395038, 24.61451220740829, 24.812533834540638, 23.893380083955623]
    ]
)

def f(x, x_max, alpha):
    """Весовая функция 𝑓(𝑥𝑖𝑗)"""
    return np.where(x <= x_max, (x/x_max)**alpha, 1)

def update_glove_weights(x, w, d, alpha, max_x, learning_rate):
    """
    x - square integer matrix VocabSize x VocabSize - coocurrence matrix
    w - VocabSize x EmbSize - first word vectors
    d - VocabSize x EmbSize - second word vectors
    alpha - float - power in weight smoothing function f
    max_x - int - maximum coocurrence count in weight smoothing function f
    learning_rate - positive float - size of gradient step
    """
    x = np.asarray(x)
    w = np.asarray(w)
    d = np.asarray(d)
    
    f_x = f(x, max_x, alpha)
    gloveloss = np.sum(f_x * (np.log(1+x) - w.dot(d.T)) ** 2)
    glove_loss_deriv = -2 * (f_x * (np.log(1+x) - w.dot(d.T)))
    
    w_grad = glove_loss_deriv.dot(d)
    d_grad = glove_loss_deriv.T.dot(w)
    
    w -= learning_rate * w_grad
    d -= learning_rate * d_grad
    return w, d

result = update_glove_weights(*sample_input)

print(np.allclose(result[0], sample_output[0]))
print(np.allclose(result[1], sample_output[1]))

True
True


## 3.4.7

Вы обучили некоторую модель эмбеддингов и получили матрицу векторов слов.

Напишите функцию, которая для заданного слова получит список наиболее похожих на него слов вместе с оценкой близости. Результат должен быть отсортирован по убыванию оценки сходства (наиболее похожие вначале). Само слово тоже должно присутствовать в выдаче.

Для оценки сходства используйте отрицательное эвклидово расстояние $sim(a, b) = - \sqrt{\sum_i {(a_i - b_i) ^ 2}}$.
Перед вычислением сходства нормируйте векторы по L2-норме.

Тесты (включая тест для примера) генерируются случайно.

Функция принимает на вход:

- embeddings - матрица размерности VocabSize x EmbSize - заранее выученная матрица векторов слов (эмбеддингов)
- query_word_id - целое неотрицательное число - идентификатор (номер) слова-запроса, для которого нужно найти наиболее похожие слова
- get_n - целое неотрицательное число - наибольшее количество похожих слов, которое нужно вернуть из функции
Функция должна возвращать список из не более чем get_n пар (номер слова, оценка сходства), отсортированных по убыванию оценки сходства со словом-запросом.

In [12]:
import sys
import ast
import numpy as np

sample_input = (
    [
        [0.7299015792584768, 0.2915364327741303, 0.5307571134639943, 0.3101345732086396, 0.8327085262119636, 0.39018382511314353, 0.678094726221033, 0.12372148102696612, 0.5966533433209616],
        [0.5411155947267721, 0.046791742239819856, 0.5358832195593092, 0.09894162419462038, 0.6350557173679914, 0.15126161842015717, 0.11375720216711405, 0.46954553941325416, 0.8281402097264261],
        [0.5323869209381028, 0.2005012376766715, 0.5925043884236925, 0.4621530177251649, 0.3886830034303448, 0.6403738184472031, 0.23320289120963578, 0.43574647265888766, 0.5305633832484254]
    ],
    0,
    8
)

sample_output = [[0.0, -0.0], [2.0, -0.502892189275798], [1.0, -0.542231021984391]]

def negative_euclidean(a, b):
    return -np.sqrt(np.sum((a-b)**2, axis=1))

def l2_norm_vector(x):
    return x / np.sqrt(np.sum(x**2, axis=1, keepdims=True))

def get_nearest(embeddings, query_word_id, get_n):
    """
    embeddings - VocabSize x EmbSize - word embeddings
    query_word_id - integer - id of query word to find most similar to
    get_n - integer - number of most similar words to retrieve

    returns list of `get_n` tuples (word_id, similarity) sorted by descending order of similarity value
    """
    embeddings = np.asarray(embeddings)
    
    # Нормирование
    embeddings = l2_norm_vector(embeddings)
    
    distances = negative_euclidean(embeddings[query_word_id], embeddings)
    
    similars = [(word, distance) for word, distance in enumerate(distances)]
    similars = sorted(similars, key=lambda pair: pair[1], reverse=True)

    return similars[:get_n]


result = get_nearest(*sample_input)

result

[(0, -0.0), (2, -0.502892189275798), (1, -0.542231021984391)]

## 3.6.6

Примените свёртку к входной последовательности, в которой каждый элемент кодируется вектором размерности 2 (часть 1)

Рассмотрим многоканальную свертку, принимающую на вход матрицу $X^{InLen, InChannels}$
 , где InLen - количество строк, равное длине входной последовательности, InChannels - количество столбцов, равное количеству признаков для каждого элемента во входной последовательности.

В результате получается матрица $Y^{OutLen}$
 , где $OutLen = InLen - K + 1$ - длина выходной последовательности, K - размер ядра свёртки.

Ядро свертки с одним выходным каналом задаётся двумерным тензором $Kernel^{InChannels, K}$

Многоканальная свёртка реализуется следующей формулой (аналогично одноканальной, но на выходной канал влияют все входные каналы):

$Y[pos] = Bias+ \sum_{k=0}^{K-1} \sum_{ic=0}^{InChannels - 1} X[pos + k,ic] Kernel[ic,k]$

Обратите внимание, что по аналогии с логистической регрессией, кроме ядра свёртки, есть ещё обучаемый параметр сдвига $Bias$.

Рассмотрим конкретный пример:

![](https://ucarecdn.com/9bfefed6-b35d-4fdc-a154-6ef680a01aaa/)
 

Для входной последовательности:

$X^{5, 2} = \left( \begin{matrix} 1 & 0 \\ 1 & 1 \\ 0 & 0 \\ 0 & 1 \\ 1 & 0 \\ \end{matrix} \right)$

Ядра, заданного следующей матрицей и нулевого смещения:

$Kernel^{2, 3}= \left( \begin{matrix} 1 & 1 & 0\\ 0 & 1 & 1\\ \end{matrix} \right), Bias=0$

Посчитайте выходной вектор YY:

$Y[pos] = Bias + \sum_{k=0}^{2} \sum_{ic=0}^{1} X[pos + k,ic] Kernel[ic, k]$

В ответ впишите содержимое вектора YY , состоящего из трех чисел, числа разделяйте пробелами. В качестве разделителя дробной части используйте точку.

Параметр сдвига считайте нулевым.

In [13]:
import numpy as np

x_in = np.array([
    [1, 0],
    [1, 1],
    [0, 0],
    [0, 1],
    [1, 0]
])

kernel = np.array([
    [1, 1, 0],
    [0, 1, 1]
])

kernel_size = kernel.shape[1]
output_len = x_in.shape[0] - kernel_size + 1
print(f'Output len: {output_len}')

for i in range(output_len):
    print(np.sum(x_in[0+i:kernel_size+i, :] * kernel.T))

Output len: 3
3
2
1


## 3.6.7

Примените свёртку к входной последовательности, в которой каждый элемент кодируется вектором размерности 2. (часть 2)

Усложним еще немного многоканальную свертку: теперь у нас будет несколько выходных каналов.

По прежнему будем подавать на вход матрицу X^{InLen, InChannels}X 
InLen,InChannels
 , где InLen - количество строк, равное длине входной последовательности, InChannels - количество столбцов, равное количеству признаков для каждого элемента во входной последовательности.

Но на этот раз ядро многоканальной свёртки задается трёхмерным тензором Kernel^{OutChannels, InChannels, K}Kernel 
OutChannels,InChannels,K
 , где новым измерением будет OutChannels - количество признаков для каждого элемента выходной последовательности.

Тогда результатом применения свертки будет матрица Y^{OutLen, OutChannels}Y 
OutLen,OutChannels
 , где OutLen = InLen - K + 1OutLen=InLen−K+1 - длина выходной последовательности, K - размер ядра свёртки, OutChannels - количество признаков для каждого элемента выходной последовательности.

Свёртка реализуется следующей формулой:

$Y[pos,oc] = Bias[oc] + \sum_{k=0}^{K-1} \sum_{ic=0}^{InChannels - 1} X[pos + k,ic] Kernel[oc,ic,k]$

Обратите внимание,  теперь параметр сдвига - это вектор обучаемых параметров $Bias^{OutChannels}$Bias 

Рассмотрим модифицированный пример из прошлого шага:

![](https://ucarecdn.com/e8d1ab37-c4c1-4ef5-bed7-f832497f4df1/)

Входная последовательность:

$X^{5, 2} =\left( \begin{matrix} 1 & 0 \\ 1 & 1 \\ 0 & 0 \\ 0 & 1 \\ 1 & 0 \\ \end{matrix} \right)$

Ядро (каждая матрица соответствует срезу по первому измерению - OutChannels, - и имеет семантику InChannels x KernelSize):

$Kernel^{2,2, 3}=\left( \begin{matrix} 1 & 1 & 0\\ 0 & 1 & 1\\ \end{matrix} \right) \left( \begin{matrix} 1 & 0 & 0\\ 0 & 0 & 1\\ \end{matrix} \right)$

В ответ впишите содержимое матрицы, каждая строка матрицы на отдельной строке, числа разделять пробелами. В качестве разделителя дробной части использовать точку.

Параметр сдвига считайте нулевым.

In [14]:
import numpy as np

x_in = np.array([
    [1, 0],
    [1, 1],
    [0, 0],
    [0, 1],
    [1, 0]
])

kernel = np.array([
    [
        [1, 1, 0],
        [0, 1, 1]
    ],
    [
        [1, 0, 0],
        [0, 0, 1]
    ]
])

kernel_size = kernel.shape[2]
out_channels = kernel.shape[0]

output_len = x_in.shape[0] - kernel_size + 1
print(f'Output len: {output_len}, out channels: {out_channels}')

for ker in kernel:
    for i in range(output_len):
        print(np.sum(x_in[i:kernel_size+i, :] * ker.T))

Output len: 3, out channels: 2
3
2
1
1
2
0


## 3.8.1

Напишите функцию, применяющую свёрточный модуль к последовательности. В качестве пособия можно использовать объяснение и формулу из задачи 3.6.7.

In [15]:
import sys
import ast
import numpy as np

sample_input = (
    [
        [0.6685863697825855, 0.9865099796400376],
        [0.32297881307708, 0.9908870158650515],
        [0.8359169063921157, 0.5443776713017927],
        [0.5363888029267118, 0.34755850459471804],
        [0.5966372342560426, 0.9834742307894673],
        [0.7371274295314912, 0.03279590013405109],
        [0.08580648148402137, 0.2850655085982621],
        [0.584942134340805, 0.7981720699451806],
        [0.19604496972304086, 0.991819073359733],
        [0.5622511488322055, 0.07928952553499002],
        [0.24152151330089744, 0.2865696384305756],
        [0.882594506643994, 0.5949729821472712],
        [0.10820233432771786, 0.8549971123651271],
        [0.18754460128195194, 0.6303661925298489],
        [0.3551051497971416, 0.9452980688158904],
        [0.6525044770663634, 0.8054232618991838]
    ],
    [
        [
            [0.8638059436915633, 0.4002290439648929, 0.8174398057982054, 0.34082478315585973, 0.5565832130592809],
            [0.08497737591188492, 0.7853885140384725, 0.1645895575029136, 0.6294907704137637, 0.8169862258229014]
        ],
        [
            [0.21527072709338957, 0.9185427760457524, 0.5167378860756242, 0.12177789993763499, 0.4201289643444214],
            [0.8389450071463863, 0.6238637143288427, 0.5098771768082815, 0.1436853463091461, 0.12036561608743845]
        ],
        [
            [0.8347050184618267, 0.12339875692133984, 0.13629086964943626, 0.623950910768403, 0.7092189295761365],
            [0.9578703072530402, 0.31612669923975534, 0.44018179806384916, 0.26615330385390035, 0.2745979551030847]
        ]
    ],
    [0.6574440445518804, 0.3253787051567585, 0.3119672663686287]
)

sample_output = (
    [
        [4.536374007375313, 3.40559782717228, 3.6420328514923983],
        [3.5379175544464094, 3.3156379278173618, 3.197855475391681],
        [3.11553432527298, 2.6461513962620473, 2.829234462423411],
        [3.955826212094638, 2.6848769190617205, 2.355474927521796],
        [3.315456250217841, 2.5537928749371677, 2.979329283784809],
        [3.2335971417998324, 1.8896105910662777, 2.297264030641381],
        [2.5503637613247276, 2.441085171765234, 2.066353981430436],
        [3.8006614791608166, 2.7637277539201435, 3.030330546451793],
        [2.8770535772488457, 2.377839924681041, 2.6997168451820808],
        [3.4854598947993667, 1.9636904181843315, 1.9609925225671008],
        [3.3707917076650484, 2.6679105220798633, 2.272378980117733],
        [4.179540206172764, 2.615786698404636, 3.36235456621979]
    ]
)

def apply_convolution(data, kernel, bias):
    """
    data - InLen x InChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutLen x OutChannels
    """
    data = np.asarray(data)
    kernel = np.asarray(kernel)
    bias = np.asarray(bias)
    
    kernel_size = kernel.shape[2]
    
    out_channels = kernel.shape[0]
    output_len = data.shape[0] - kernel_size + 1
    print(f'Output len: {output_len}, out channels: {out_channels}')
    
    output = np.zeros(shape=(output_len, out_channels))

    for kernel_i, ker in enumerate(kernel):
        for i in range(output_len):
            output[i:kernel_size+i, kernel_i] = bias[kernel_i] + np.sum(data[i:kernel_size+i, :] * ker.T)
    return output


result = apply_convolution(*sample_input)

result

Output len: 12, out channels: 3


array([[4.53637401, 3.40559783, 3.64203285],
       [3.53791755, 3.31563793, 3.19785548],
       [3.11553433, 2.6461514 , 2.82923446],
       [3.95582621, 2.68487692, 2.35547493],
       [3.31545625, 2.55379287, 2.97932928],
       [3.23359714, 1.88961059, 2.29726403],
       [2.55036376, 2.44108517, 2.06635398],
       [3.80066148, 2.76372775, 3.03033055],
       [2.87705358, 2.37783992, 2.69971685],
       [3.48545989, 1.96369042, 1.96099252],
       [3.37079171, 2.66791052, 2.27237898],
       [4.17954021, 2.6157867 , 3.36235457]])

## 3.8.2

Вы применяете свёрточный модуль к данным: $y = convolve(x, kernel, bias)$, где $x \in \mathbb{R}^{InLen \times InChannels}$ - входная последовательность, $kernel \in \mathbb{R}^{OutChannels \times InChannels \times KernelSize}$ - ядро свёртки, $bias \in \mathbb{R}^{OutChannels}$ - параметры сдвига для каждого выходного канала.

Напишите функцию, которая находит значение производной результата по ядру свёртки: $\frac{\partial y} {\partial kernel}$

In [16]:
import sys
import ast
import numpy as np

sample_input = (
    [
        [0.1559846921787793, 0.31890243279158936, 0.4294841981370352],
        [0.6287087193276831, 0.041166388481120975, 0.0670771539905104],
        [0.15852860049868056, 0.5535509628878403, 0.7578893631796608],
        [0.505354018460584, 0.39104507392557886, 0.267830936523598],
        [0.35352084058390487, 0.09557605719492113, 0.17762289879326898],
        [0.17895124947325458, 0.2038143268404633, 0.45431038892117714],
        [0.44910520386366004, 0.28874952266426823, 0.48880576852948343],
        [0.978917156152191, 0.11927306276379035, 0.6831784491543058], 
        [0.7665547409895508, 0.02661305420346527, 0.662020788170643]
    ],
    [
        [0.9423069699375323, 2.235723993887441],
        [1.051430354504024, 2.5441990558270864],
        [1.3060781439427196, 2.77617352972661],
        [1.1097986223660692, 2.019161891632857],
        [0.8312656308401454, 2.1968468090151005],
        [1.0883546513743934, 3.182136137325698],
        [1.4545203460970286, 3.3294233853738975]
    ],
    [
        [
            [0.7285150274194675, 0.13990616439149894, 0.08385531710791316],
            [0.8119118106104425, 0.19272155988045991, 0.010762309371285528],
            [0.5242309485683324, 0.33106798748722055, 0.2219888201243384]
        ],
        [
            [0.31261137319823096, 0.3951341806652614, 0.954412244770657],
            [0.5475239344122861, 0.6407966544293683, 0.2840031545245296],
            [0.9267337670407934, 0.626334029479077, 0.4315268897320006]
        ]
    ],
    [0.03900532471824536, 0.6619593919342232]
)

sample_output = (
    [
        [
            [2.4301533243865463, 3.253085788359958, 3.3909318100218258],
            [1.8928047647857822, 1.6931753947579833, 1.6786220604803275],
            [2.643020708074734, 2.8967149590920043, 3.491658593272137]
        ],
        [
            [2.4301533243865463, 3.253085788359958, 3.3909318100218258],
            [1.8928047647857822, 1.6931753947579833, 1.6786220604803275],
            [2.643020708074734, 2.8967149590920043, 3.491658593272137]
        ]
    ]
)

def calculate_kernel_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """
    
    x = np.asarray(x)
    y = np.asarray(y)
    kernel = np.asarray(kernel)
    bias = np.asarray(bias)
    
    dy_dkernel = np.zeros(shape=kernel.shape)
    
    kernel_size = kernel.shape[2]
    output_len = y.shape[0]
    
    for kernel_i, ker in enumerate(kernel):
        helper = np.zeros(shape=ker.T.shape)
        for i in range(output_len):
            helper += x[i:kernel_size+i, :]
        dy_dkernel[kernel_i] = helper.T
    
    return dy_dkernel

result = calculate_kernel_grad(*sample_input)

result

array([[[2.43015332, 3.25308579, 3.39093181],
        [1.89280476, 1.69317539, 1.67862206],
        [2.64302071, 2.89671496, 3.49165859]],

       [[2.43015332, 3.25308579, 3.39093181],
        [1.89280476, 1.69317539, 1.67862206],
        [2.64302071, 2.89671496, 3.49165859]]])

## 3.8.3

Вы применяете свёрточный модуль к данным: $y = convolve(x, kernel, bias)$, где $x \in \mathbb{R}^{InLen \times InChannels}$ - входная последовательность, $kernel \in \mathbb{R}^{OutChannels \times InChannels \times KernelSize}$ - ядро свёртки, $bias \in \mathbb{R}^{OutChannels}$ - параметры сдвига для каждого выходного канала.

Напишите функцию, которая находит значение производной результата по входу: $\frac{\partial y} {\partial x}$

In [17]:
import sys
import ast
import numpy as np

sample_input = (
    [
        [0.5031766517322117, 0.30744410216949514],
        [0.04690208449415345, 0.322727131626243],
        [0.1388690574185909, 0.48576543724022325],
        [0.5260018011862109, 0.5859221562109312], 
        [0.9194272143904142, 0.3887293155713266],
        [0.26873714217871125, 0.9546207791313607],
        [0.8974007607375208, 0.5713329992292489], 
        [0.378989716528242, 0.49787928388753266]
    ],
    [
        [1.5157583762374225, 0.9460413662192456, 0.9802340338281511],
        [1.5728362445918327, 0.996409724139607, 1.2530013664472253],
        [1.9068174476481374, 1.430592927945995, 1.6704630594015581],
        [2.189768979209843, 2.3149543871163503, 2.1601629609824995],
        [2.8353457102707083, 1.7422359297539565, 1.816707087141475],
        [2.0532913525958474, 1.9924093441385802, 2.3069493556139014]
    ],
    [
        [
            [0.8077620147648772, 0.006392942850116379, 0.6080212915877307],
            [0.6288229869798402, 0.6410664904844843, 0.75419330562945]
        ],
        [
            [0.5355186530459589, 0.9211024178840701, 0.27725553497982014],
            [0.4507098181629161, 0.081570594016668, 0.8234980185346139]
        ],
        [
            [0.0325944131753374, 0.7744753133142763, 0.05946983249285043],
            [0.7059580971549311, 0.7969953841197822, 0.5257810951530107]
        ]
    ],
    [0.2579976950685653, 0.029957050945287222, 0.18958928880952108]
)

sample_output = (
    [
        [1.3758750809861735, 1.7854909022976875],
        [3.0778457550346365, 3.305123370918622],
        [4.022592414095037, 5.4085957902356965],
        [4.022592414095037, 5.4085957902356965],
        [4.022592414095037, 5.4085957902356965],
        [4.022592414095037, 5.4085957902356965],
        [2.646717333108864, 3.623104887938009],
        [0.9447466590604012, 2.1034724193170744]
    ]
)

def calculate_conv_x_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns InLen x InChannels
    """
    
    x = np.asarray(x)
    y = np.asarray(y)
    kernel = np.asarray(kernel)
    bias = np.asarray(bias)
    
    dy_dx = np.zeros(shape=x.shape)
    
    kernel_size = kernel.shape[2]
    output_len = y.shape[0]
    
    for kernel_i, ker in enumerate(kernel):
        for i in range(output_len):
            dy_dx[i:kernel_size+i, :] += ker.T
    
    return dy_dx

result = calculate_conv_x_grad(*sample_input)

result

array([[1.37587508, 1.7854909 ],
       [3.07784576, 3.30512337],
       [4.02259241, 5.40859579],
       [4.02259241, 5.40859579],
       [4.02259241, 5.40859579],
       [4.02259241, 5.40859579],
       [2.64671733, 3.62310489],
       [0.94474666, 2.10347242]])

### Заметка

В прошлых двух примерах главное учесть сколько раз используется ядро или i,j элемент входного тензора, чтобы понять сколько раз нужно суммировать, в последнем случае получается суммирование столько раз, сколько в данном i,j элементе происходит домножение на ядро при прямом проходе

## 3.8.4

Вы разрабатываете нейросеть, состоящую из последовательности свёрточных слоёв с разными параметрами:

 - $kernel >= 1, kernel \% 2 == 1$ - размер ядра свёртки (обязательно нечётный)
 - $dilation >= 1$ - коэффициент прореживания или расстояние между позициями элементов, учитываемых одной свёрткой (если dilation = 1, то это "плотная" свёртка, если dilation = 2, пропускается каждый второй элемент)

Напишите функцию для расчёта ширины рецептивного поля - наибольшего расстояния между элементами входной последовательности (+1), которые влияют на один и тот же элемент выходной последовательности.

In [18]:
import ast
import sys
import collections
import numpy as np

kernels, dilations = [9, 9, 3], [2, 3, 4]
sample_output = 49

LayerInfo = collections.namedtuple('LayerInfo', ('kernel_size', 'dilation'))


def calculate_receptive_field(layers):
    """
    layers - list of LayerInfo

    returns int - receptive field size
    """
    r_out = 1 + layers[0].dilation * (layers[0].kernel_size - 1)
    for i in range(1, len(layers)):
        r_out = r_out + layers[i].dilation * (layers[i].kernel_size - 1)
    return r_out


layers = [LayerInfo(k, d) for k, d in zip(kernels, dilations)]
print(layers[0].dilation)

result = calculate_receptive_field(layers)
print(result)

2
49


In [19]:
1.1 ** 99

12527.829399838527

## 4.4 Механизм внимания

Вспомните основную формулу из предыдущей задачи с глобальным avg-пулингом:

 
$avg(Input) = \frac{1}{k} \sum_{i=0}^{k-1} Input[i]$

Технически, основное отличие механизма внимания от avg-пулинга - наличие весов при слагаемых: 
$Attention[Ch] = \sum_{i=0}^{InLen-1} AttScores[i] \cdot Input[i,Ch]$

где $Input \in \mathbb{R} ^ {InLen \times EmbSize}$ - матрица признаков входной последовательности, $Ch$ - номер столбца (номер признака), $AttScores[i]$ - оценка значимости для i-го элемента входной последовательности ($AttScores \in \mathbb{R}^{InLen}$, $0 \le AttScores[i] \le 1$).

Примените механизм внимания к входной матрице

$Input = \left( \begin{matrix} 1 & 0 & 2 \\ 0 & 1 & 3 \\ 1 & 3 & 0 \\ 0 & 0 & 0\end{matrix} \right) \in \mathbb{R} ^ {InLen \times EmbSize}$

с учётом оценок значимости $AttScores = \left( \begin{matrix} 0.1 & 0.5 & 0.3 & 0.1 \end{matrix} \right)$

Результат запишите в виде вектора на одной строке, элементы разделяйте пробелами. В качестве десятичного разделителя используйте точку.

In [20]:
import numpy as np
input_matrix = np.array([
    [1, 0, 2],
    [0, 1, 3],
    [1, 3, 0],
    [0, 0, 0]
])
attention = np.array([[0.1, 0.5, 0.3, 0.1]])

np.sum(input_matrix * attention.T, axis=0)

array([0.4, 1.4, 1.7])

Рассмотрим механизм внимания, в котором релевантности элементов входной последовательности расчитываются с помощью скалярного произведения с фиксированным вектором-запросом.

Как и раньше, $Input \in \mathbb{R} ^ {InLen \times EmbSize}$ - входная матрица.

Релевантности элементов входной последовательности можно посчитать с помощью матричного произведения $UnnormScores = Input \cdot Query$, где $Query \in \mathbb{R}^{EmbSize}$ - вектор-запрос, характеризующий значимость признаков. Тогда $UnnormScores \in \mathbb{R}^{InLen}$ - вектор релевантностей элементов входной последовательности (чем больше $UnnormScores[i]$, тем значимее i-ый элемент).

Затем релевантности нормируют: $AttScores = Softmax(UnnormScores)$.

Ну и наконец, результат операции внимания считается по формуле из предыдущей задачи $Attention[Ch] = \sum_{i=0}^{InLen-1} Input[i,Ch] \cdot AttScores[i]$

Примените механизм внимания к входной матрице

$Input = \left( \begin{matrix} 1 & 0 & 2 \\ 0 & 1 & 3 \\ 1 & 3 & 0 \\ 0 & 0 & 0\end{matrix} \right)$
 
с учётом вектора-запроса $Query = \left( \begin{matrix} 0 & 0 & 1 \end{matrix} \right)$

Результат запишите в виде вектора на одной строке, элементы разделяйте пробелами. В качестве десятичного разделителя используйте точку. Числа округлите до не менее чем двух знаков после запятой.

In [21]:
import numpy as np
input_matrix = np.array([
    [1, 0, 2],
    [0, 1, 3],
    [1, 3, 0],
    [0, 0, 0]
])

def softmax(z, axis=1):
    if axis == 1:
        exp_sum = np.sum(np.exp(z), axis=1).reshape(-1, 1)
    elif axis == 0:
        exp_sum = np.sum(np.exp(z), axis=0)
    else:
        exp_sum = np.sum(np.exp(z))
    return np.exp(z) / exp_sum

query = np.array([0, 0, 1])
unnorm_scores = input_matrix.dot(query)
attention = softmax(unnorm_scores[np.newaxis, :])

result = np.sum(input_matrix * attention.T, axis=0)

print(f' '.join(map(str, np.round(result, 2))))

0.28 0.78 2.55


## 4.5 Механизм самовнимания

Механизм self-attention принимает на вход матрицу $Input \in \mathbb{R} ^ {InLen \times EmbSize}$, где $InLen$ - количество строк, соответствующее длине входной последовательности, а $EmbSize$ - количество столбцов, соответствующее количеству признаков для каждого элемента.

Найдите матрицу попарного сходства элементов $Logits=Input \cdot Input^T$

Входная матрица имеет следующий вид

$Input = \left( \begin{matrix} 1 & 0 \\ 0 & 1 \\ 1 & 1 \\ 0 & 0\end{matrix} \right)$
 
Результат запишите в виде матрицы, на одной строке - элементы одной строки матрицы, разделённые пробелами.

Нужные операции можно выполнить в numpy, используя функцию dot или matmul, а также оператор матричного умножения a @ b.

In [22]:
import numpy as np

input_matrix = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [0, 0]
])

logits = input_matrix.dot(input_matrix.T)
logits

array([[1, 0, 1, 0],
       [0, 1, 1, 0],
       [1, 1, 2, 0],
       [0, 0, 0, 0]])

Используя матрицу $Logits$, полученную в результате решения предыдущей задачи, найдите выходную матрицу $Result \in \mathbb{R}^{InLen \times EmbSize}$ для механизма self-attention.

Для этого нужно

 - матрицу $Logits$ нормировать с помощью softmax по строкам $AttScores = softmax(Logits, rows)$, в результате $0 \leq AttScores[i,j] \leq 1$ и $\sum_{j=0}^{InLen-1} AttScores[i, j] = 1$
 - найти взвешенную сумму исходных признаков с учётом найденых весов: $Result=AttScores \cdot Input$ (с помощью матричного произведения)
 
Результат запишите в виде матрицы, на одной строке - элементы одной строки матрицы, разделённые пробелами. В качестве десятичного разделителя используйте точку. Ответ округлите до не менее чем двух знаков после запятой.

In [23]:
attscore = softmax(logits, axis=1)
result = attscore.dot(input_matrix)
print(' '.join(map(str, np.round(result.ravel(), 2))))

0.73 0.5 0.5 0.73 0.73 0.73 0.5 0.5


In [24]:
result

array([[0.73105858, 0.5       ],
       [0.5       , 0.73105858],
       [0.73105858, 0.73105858],
       [0.5       , 0.5       ]])

Давайте теперь посмотрим, как работает более общий вариант self-attention - когда в качестве ключей, запросов и значений используются разные матрицы.

На вход мы получаем всё ту же матрицу

$Input = \left( \begin{matrix} 1 & 0 \\ 0 & 1 \\ 1 & 1 \\ 0 & 0\end{matrix} \right)$

Общий алгоритм состоит из следующих основных шагов:

- Найти значения ключей, запросов и значений, используя линейное преобразование 

$Keys = Input \cdot Proj_K + Bias_K$

$Queries = Input \cdot Proj_Q + Bias_Q$

$Values = Input \cdot Proj_V + Bias_V$

- Найти матрицу попарного сходства, используя полученные матричное произведение запросов и ключей 

$Logits=Queries \cdot Keys^T$

- Найти коэффициенты усреднения, нормировав матрицу попарного сходства с помощью softmax по строкам

$AttScores = softmax(Logits, rows)$

- Найти результат с помощью матричного произведения матриц значений и коэффициентов 

$Result=AttScores \cdot Values$

Вам требуется найти значение матрицы $Result \in \mathbb{R}^{InLen \times EmbSize}$ с учётом следующих параметров преобразования:

$Proj_K = \left(\begin{matrix}1 & 0 \\ 0 & 0\end{matrix} \right)$

$Proj_Q = \left(\begin{matrix}0 & 0 \\ 1 & 0\end{matrix} \right)$

$Proj_V = \left(\begin{matrix}1 & 0 \\ 0 & 1\end{matrix} \right)$

$Bias_K = Bias_Q = Bias_V = \left(\begin{matrix}0 & 0\end{matrix}\right)$

Результат запишите в виде матрицы, на одной строке - элементы одной строки матрицы, разделённые пробелами. В качестве десятичного разделителя используйте точку. Ответ округлите до не менее чем двух знаков после запятой.

In [25]:
import numpy as np

input_matrix = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [0, 0]
])

proj_k = np.array([
    [1, 0],
    [0, 0]
])

proj_q = np.array([
    [0, 0],
    [1, 0]
])

proj_v = np.array([
    [1, 0],
    [0, 1]
])

bias_k = np.array([0, 0])
bias_q = np.array([0, 0])
bias_v = np.array([0, 0])

keys = input_matrix.dot(proj_k) + bias_k
queries = input_matrix.dot(proj_q) + bias_q
values = input_matrix.dot(proj_v) + bias_v

logits = queries.dot(keys.T)
attscore = softmax(logits, axis=1)

result = attscore.dot(values)
print(result)
print(' '.join(map(str, np.round(result.ravel(), 2))))

[[0.5        0.5       ]
 [0.73105858 0.5       ]
 [0.73105858 0.5       ]
 [0.5        0.5       ]]
0.5 0.5 0.73 0.5 0.73 0.5 0.5 0.5


Ещё один шаг - перейдём от простого self-attention к multihead self-attention.

Общий алгоритм - точно такой же, как и в предыдущей задаче. Отличие в том, что нам нужно несколько раз применить механизм внимания с разными параметрами преобразований $Result^i = SelfAttention(Input, Proj^i_K, Proj^i_Q, Proj^i_V)$, $Result^i \in \mathbb{R}^{InLen \times \frac{EmbSize} {HeadsN}}$

Результат $MHResult \in \mathbb{R} ^ {InLen \times EmbSize}$ получается конкатенацией $Result^i$ по столбцам: $MHResult = \left[ Result^1, Result^2, ..., Result^{HeadsN} \right]$

**Вам требуется найти $MHResult$ для входных данных**

$Input = \left( \begin{matrix} 1 & 0 \\ 0 & 1 \\ 1 & 1 \\ 0 & 0\end{matrix} \right)$
 
**с учётом количества "голов" $HeadsN = 2$ и параметров преобразований**

$Proj^1_K = \left(\begin{matrix}1 & 0 \\ 0 & 0\end{matrix} \right)$,

$Proj^2_K = \left(\begin{matrix}0 & 0 \\ 1 & 0\end{matrix} \right)$,

$Proj^1_Q = \left(\begin{matrix}0 & 1 \\ 1 & 0\end{matrix} \right)$,

$Proj^2_Q = \left(\begin{matrix}1 & 1 \\ 1 & 1\end{matrix} \right)$,

$Proj^1_V = \left(\begin{matrix}1 \\ 0\end{matrix} \right)$,

$Proj^2_V = \left(\begin{matrix}0 \\ 1\end{matrix} \right)$

$Bias^i_K = Bias^i_Q = \left(\begin{matrix}0 & 0\end{matrix}\right), \quad Bias^i_V = 0$

Результат запишите в виде матрицы, на одной строке - элементы одной строки матрицы, разделённые пробелами. В качестве десятичного разделителя используйте точку. Ответ округлите до не менее чем двух знаков после запятой.

In [26]:
import numpy as np

N = 2

input_matrix = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [0, 0]
])

proj_k = (
    np.array([
        [1, 0],
        [0, 0]
    ]),
    np.array([
        [0, 0],
        [1, 0]
    ])
)

proj_q = (
    np.array([
        [0, 1],
        [1, 0]
    ]),
    np.array([
        [1, 1],
        [1, 1]
    ])
)

proj_v = (
    np.array([
        [1],
        [0]
    ]),
    np.array([
        [0],
        [1]
    ])
)

bias_k = (
    np.array([0, 0]),
    np.array([0, 0])
)

bias_q = (
    np.array([0, 0]),
    np.array([0, 0])
)

bias_v = (0, 0)

result = []
for i in range(N):
    keys = input_matrix.dot(proj_k[i]) + bias_k[i]
    queries = input_matrix.dot(proj_q[i]) + bias_q[i]
    values = input_matrix.dot(proj_v[i]) + bias_v[i]

    logits = queries.dot(keys.T)
    attscore = softmax(logits, axis=1)
    
    semiresult = attscore.dot(values)
    result.append(semiresult)
    
result = np.concatenate(result, axis=1)

print(result)
print(' '.join(map(str, np.round(result.ravel(), 2))))

[[0.5        0.73105858]
 [0.73105858 0.73105858]
 [0.73105858 0.88079708]
 [0.5        0.5       ]]
0.5 0.73 0.73 0.73 0.73 0.88 0.5 0.5


## 4.7 Теоретические вопросы: Модель языка и трансформеры

Напишите функцию, реализующую max-pooling с заданной шириной окна. В качестве пособия можно использовать [другую задачу](https://stepik.org/lesson/262248/step/2?unit=243131).

Функция должна возвращать два тензора одинаковой размерности $OutLen \times EmbSize$:

- первый тензор - основной результат - результат применения max-пулинга к каждому столбцу для каждой позиции скользящего окна
- второй тензор - информация, нужная для расчёта градиента, - относительные индексы максимальных элементов для каждого столбца для каждой позиции скользящего окна

Например, для фрагмента входной матрицы $\left( \begin{matrix}1 & 0 & 3 \\ 0 & 1 & 4\end{matrix}\right)$ и размера скользящего окна $k=2$ первый результат должен иметь вид $result = \left(\begin{matrix} 1 & 1 & 4 \end{matrix}\right)$, а второй 
$indices = \left( \begin{matrix} 0 & 1 & 1\end{matrix} \right)$. Индексы - номера строк относительно позиции окна (
$0 \leq indices_i < k$), из которых были взяты соответствующие элементы $result$

In [31]:
import numpy as np

sample_input = (
    [
        [0.6046018907385543, 0.0812964077275945, 0.6366439552273822, 0.20134327995534496],
        [0.8106187774962709, 0.4395507898340978, 0.8290096270213004, 0.05773841312522798],
        [0.938964520620817, 0.3860407857274528, 0.21318174478828456, 0.07860176987690048],
        [0.04840110723428537, 0.6411287103553837, 0.509253427569025, 0.7094441369109541],
        [0.691552879511939, 0.4979735285634219, 0.07060470682483455, 0.7631262538014161]
    ],
    2
)

sample_output = (
    [
        [0.8106187774962709, 0.4395507898340978, 0.8290096270213004, 0.20134327995534496],
        [0.938964520620817, 0.4395507898340978, 0.8290096270213004, 0.07860176987690048],
        [0.938964520620817, 0.6411287103553837, 0.509253427569025, 0.7094441369109541],
        [0.691552879511939, 0.6411287103553837, 0.509253427569025, 0.7631262538014161]
    ],
    [
        [1.0, 1.0, 1.0, 0.0],
        [1.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 1.0, 1.0],
        [1.0, 0.0, 0.0, 1.0]
    ]
)

def max_pooling(features, kernel_size):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window

    returns tuple of two matrices of shape OutLen x EmbSize:
         - output features (main result)
         - relative indices of maximum elements for each position of sliding window
    """
    features = np.asarray(features)
    
    out_len = features.shape[0] - kernel_size + 1
    emb_size = features.shape[1]
    
    result = np.zeros((out_len, emb_size))
    indices = np.zeros((out_len, emb_size))
    
    for i in range(out_len):
        for j in range(emb_size):
            result[i, j] = np.max(features[i:i+kernel_size, j])
            indices[i, j] = np.argmax(features[i:i+kernel_size, j])
    
    return result, indices


result, indices = max_pooling(*sample_input)

result, indices

(array([[0.81061878, 0.43955079, 0.82900963, 0.20134328],
        [0.93896452, 0.43955079, 0.82900963, 0.07860177],
        [0.93896452, 0.64112871, 0.50925343, 0.70944414],
        [0.69155288, 0.64112871, 0.50925343, 0.76312625]]),
 array([[1., 1., 1., 0.],
        [1., 0., 0., 1.],
        [0., 1., 1., 1.],
        [1., 0., 0., 1.]]))

Прямому проходу по модулю max-пулинга была посвящена предыдущая задача.

Теперь займёмся обратным проходом: напишите функцию, вычисляющую производную функции потерь по входам модуля max-пулинга $\frac{\partial Loss}{\partial features}$.

Функция принимает следующие аргументы:

- $features \in \mathbb{R}^{InLen \times EmbSize}$ - признаки, которые были переданы на вход модулю при прямом проходе
- $2 \leq kernel\_size \leq InLen$ - размер скользящего окна
- $indices \in \mathbb{N}^{OutLen \times EmbSize}, \quad 0 \leq indices < kernel\_size$ - относительная позиция максимального элемента внутри скользящего окна для каждого элемента выходного тензора (смещение относительно номера строки)
- $dldout = \frac{\partial Loss} {\partial out} \in \mathbb{R}^{OutLen \times EmbSize}$ - значение производной функции потерь по выходам слоя max-пулинга (то есть производная по входам следующего слоя)

Вам может быть полезна формула производной кусочно-линейной функции 
$ReLU(x) = \begin{cases} 0 & if \quad x \leq 0 \\ x & otherwise \end{cases}$; $\frac{\partial ReLU(x)} {\partial x} = \begin{cases} 0 & if \quad x \leq 0 \\ 1 & otherwise \end{cases}$

А также помните про правило цепочки: 
$\frac{\partial Loss} {\partial features} = \frac{\partial Loss} {\partial out} \frac{\partial out} {\partial features}$

In [44]:
import numpy as np

sample_input = (
    [
        [-1.2420542766989977, -0.045100789663994285, 1.858151857421511, 0.10732741246325356],
        [-1.480497780371414, -0.12486054931133332, -0.18422425981847368, -1.4228130362490647],
        [-0.8417536968892625, 0.9802583655274091, -0.18413492661665792, -1.5582607186399924],
        [1.325799250424393, 0.08149768959330334, -1.454876921308986, 0.1408031456023352],
        [0.1637602967235608, -0.21250114632967532, 0.8362859721448469, 0.717774697701287],
        [-0.7641399532198978, -2.112568530488304, 0.20121440705964902, 0.015624280892385661],
        [1.3862200103422582, 0.6508694196448389, -1.162417318743681, 1.5202488401790915],
        [1.3947418297193952, -1.013483406336198, -2.0608332074129545, -1.733019236247151],
        [1.0932612618870112, 0.8071262618398916, 0.15924519176972282, -0.6885825807454318]
    ],
    6,
    [
        [3.0, 2.0, 0.0, 4.0],
        [5.0, 1.0, 3.0, 5.0],
        [5.0, 0.0, 2.0, 4.0],
        [4.0, 5.0, 1.0, 3.0]
    ],
    [
        [-0.0763791951131031, -0.8729161683329371, 0.7454337173675266, -2.508470377801969],
        [-0.9080189656976042, 0.6952579391985969, 0.2829942797947518, 0.35396918585149195],
        [0.339009358836277, -1.9496823733556254, -0.11017174942549533, 1.4591363247582954],
        [-1.1161622731011638, -2.371055190136431, -1.174384738333498, 0.4672192180206214]
    ]
)

sample_output = (
    [
        [0.0, 0.0, 0.7454337173675266, 0.0],
        [0.0, 0.0, 0.0, 0.0],
        [0.0, -2.1273406024899657, 0.0, 0.0],
        [-0.0763791951131031, 0.0, 0.0, 0.0],
        [0.0, 0.0, -1.0015622079642417, -2.508470377801969],
        [0.0, 0.0, 0.0, 0.0],
        [-0.9080189656976042, 0.0, 0.0, 2.2803247286304087],
        [-0.7771529142648868, 0.0, 0.0, 0.0],
        [0.0, -2.371055190136431, 0.0, 0.0]
    ]
)

def max_pooling_dldfeatures(features, kernel_size, indices, dldout):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window
    indices - OutLen x EmbSize - relative indices of maximum elements for each window position
    dldout - OutLen x EmbSize - partial derivative of loss function with respect to outputs of max_pooling layer

    returns InLen x EmbSize
    """
    features = np.asarray(features)
    indices = np.asarray(indices)
    dldout = np.asarray(dldout)
    
    inlen, emb_size = features.shape
    out_len = features.shape[0] - kernel_size + 1
    
    dldfeatures = np.zeros((inlen, emb_size))
    
    for i in range(out_len):
        for j in range(emb_size):
            index = int(indices[i, j])
            dldfeatures[i:i+kernel_size, j][index] += dldout[i, j]
    
    return dldfeatures


dldfeatures = max_pooling_dldfeatures(*sample_input)

dldfeatures

array([[ 0.        ,  0.        ,  0.74543372,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -2.1273406 ,  0.        ,  0.        ],
       [-0.0763792 ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.00156221, -2.50847038],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [-0.90801897,  0.        ,  0.        ,  2.28032473],
       [-0.77715291,  0.        ,  0.        ,  0.        ],
       [ 0.        , -2.37105519,  0.        ,  0.        ]])

Вектор-функция 
$softmax(x) = \left( \begin{matrix} \frac{e^{x_1}}{\sum_j e^{x_j}} & ... & \frac{e^{x_n}}{\sum_j e^{x_j}} \end{matrix} \right)$ - популярный способ нормировать вектор чисел 
$x \in \mathbb{R}^n$ так, чтобы $0 \leq softmax_i(x) \leq 1$ и $\sum_i softmax_i(x) = 1$

Напишите функцию, вычисляющую softmax для заданного вектора.

Используйте экспоненту из numpy: np.exp(x).

In [49]:
import numpy as np

sample_input = (
    [
        0.7903253367110061,
        0.1738679257213426,
        0.6840758121402977, 
        -1.4921922753864911,
        -0.20701526564877176,
        -0.13343908330179777,
        0.27078275189785883, 
        -0.47987385916752834,
        1.762361457920409,
        -0.6382574781276095,
        -1.476682298043406,
        0.13308403857533435,
        0.08629164346752129, 
        -0.15274120983311792,
        0.03422761142701722,
        2.0977915122558075, 
        -0.09695813983037735,
        1.145554587286743
    ]
)

sample_output = (
    [
        0.06860598204389033,
        0.03703718180652343,
        0.06169051603553571,
        0.006999663809155501,
        0.02530593948353462, 
        0.02723806143745914,
        0.040806327204274295,
        0.019262891730251305,
        0.18134764032840614, 
        0.01644130749673833,
        0.007109074723334883,
        0.03555704949423753, 
        0.033931576448048006,
        0.0267173505099923,
        0.032210162471156406,
        0.25362223807297396, 
        0.028250079060620836,
        0.09786695784386741
    ]
)

def softmax(x):
    """
    x - vector of n elements - input

    returns vector of n elements - softmax output
    """
    exp = np.exp(x)
    return exp / np.sum(exp, axis=0)

result = softmax(sample_input)
result

array([0.06860598, 0.03703718, 0.06169052, 0.00699966, 0.02530594,
       0.02723806, 0.04080633, 0.01926289, 0.18134764, 0.01644131,
       0.00710907, 0.03555705, 0.03393158, 0.02671735, 0.03221016,
       0.25362224, 0.02825008, 0.09786696])

Напишите функцию, реализующую производную выхода softmax по входам $\frac{\partial softmax(x)} {\partial x}$.

Помните, что и $softmax$ и $x$ - вектора из $n$ элементов, поэтому производная - это матрица, в ij-ячейке которой стоит производная i-го элемента 
$softmax_i(x)$ по j-му входу $x_j$ (i соответствует номеру строки, j - номер столбца). Такая матрица ещё называется матрицей Якоби.

Подсказка: возможно, будет проще решать эту задачу, рассматривая два случая, когда $i=j$ и когда $i \neq j$.

In [56]:
import numpy as np

sample_input = [
    -0.36170314084137395,
    1.531638983431016,
    -1.7131284538840788,
    -0.9027503682845508,
    -0.8591376176087115,
    0.16481576122888014,
    0.2286590883015934,
    0.4686776665093307,
    -0.4880318948728026,
    0.13865483857501165,
    -1.0740873577508447,
    -1.1693607815929845,
    0.6388250392697977
]

sample_output = [
    [
        0.04520990642005896,
        -0.01496136384662353,
        -0.0005831584750616379,
        -0.0013113823146393676, 
        -0.001369840806658998, 
        -0.0038138833133273915, 
        -0.004065315035397694,
        -0.005168124298133092, 
        -0.0019853599968400696,
        -0.0037154023993240717,
        -0.0011048889071803723, 
        -0.0010044813810742047, 
        -0.006126705645798535
    ],
    [
        -0.01496136384662353,
        0.2158579197223101, 
        -0.0038730635991859903,
        -0.008709582942971316, 
        -0.00909783668048341,
        -0.02533001450562798, 
        -0.026999905439354373, 
        -0.03432424452555928,
        -0.013185824889586901,
        -0.024675950714133716,
        -0.007338151103938566, 
        -0.006671291663363668, 
        -0.04069068981148138
    ],
    [
        -0.0005831584750616379,
        -0.0038730635991859903,
        0.012135730472354804,
        -0.0003394788843793973, 
        -0.0003546120941472776, 
        -0.000987303883778268,
        -0.0010523922714690001,
        -0.0013378776360475466,
        -0.0005139521780146152, 
        -0.0009618100285956132, 
        -0.0002860237242683219, 
        -0.00026003112503520596,
        -0.0015860265723719278
    ],
    [
        -0.0013113823146393676,
        -0.008709582942971316,
        -0.0003394788843793973,
        0.026866394590907443, 
        -0.0007974368009876185,
        -0.0022202075554586314, 
        -0.0023665755911748154, 
        -0.003008563102643473, 
        -0.0011557540971131601,
        -0.0021628780434165113, 
        -0.0006431981521542989,
        -0.0005847470854143117,
        -0.003566590020554541
    ],
    [
        -0.001369840806658998, 
        -0.00909783668048341, 
        -0.0003546120941472776,
        -0.0007974368009876185, 
        0.02802849045258695, 
        -0.0023191794450546825,
        -0.002472072240600495, 
        -0.003142678120181134, 
        -0.0012072750310990093,
        -0.0022592943115246175,
        -0.0006718704879216859, 
        -0.000610813803292534,
        -0.0037255806306354905
    ],
    [
        -0.0038138833133273915,
        -0.02533001450562798,
        -0.000987303883778268,
        -0.0022202075554586314,
        -0.0023191794450546825,
        0.07389852776941823, 
        -0.006882693975777504,
        -0.008749781422376342,
        -0.0033612709398950754,
        -0.006290281931106346,
        -0.0018706083437909523,
        -0.0017006155464219198,
        -0.010372686906803137
    ],
    [
        -0.004065315035397694,
        -0.026999905439354373,
        -0.0010523922714690001,
        -0.0023665755911748154,
        -0.002472072240600495,
        -0.006882693975777504,
        0.07831657234824194, 
        -0.009326614122810346,
        -0.0035828640174308843,
        -0.006704971183060923, 
        -0.0019939289172219475, 
        -0.0018127292794043132,
        -0.011056510274539641
    ],
    [
        -0.005168124298133092, 
        -0.03432424452555928,
        -0.0013378776360475466,
        -0.003008563102643473,
        -0.003142678120181134, 
        -0.008749781422376342, 
        -0.009326614122810346, 
        0.09703166928519828, 
        -0.004554797457063466,
        -0.008523847275730119,
        -0.002534827533934738,
        -0.002304473368792744,
        -0.014055840421926011
    ],
    [
        -0.0019853599968400696, 
        -0.013185824889586901,
        -0.0005139521780146152,
        -0.0011557540971131601,
        -0.0012072750310990093, 
        -0.0033612709398950754, 
        -0.0035828640174308843,
        -0.004554797457063466, 
        0.040080235966119454, 
        -0.0032744772424536947,
        -0.0009737662823977987,
        -0.0008852746134293082,
        -0.005399619220795473
    ],
    [
        -0.0037154023993240717,
        -0.024675950714133716,
        -0.0009618100285956132,
        -0.0021628780434165113,
        -0.0022592943115246175, 
        -0.006290281931106346,
        -0.006704971183060923,
        -0.008523847275730119,
        -0.0032744772424536947,
        0.07215276857928996, 
        -0.001822306074344211,
        -0.001656702778353048,
        -0.010104846597247093
    ],
    [
        -0.0011048889071803723, 
        -0.007338151103938566,
        -0.0002860237242683219,
        -0.0006431981521542989,
        -0.0006718704879216859, 
        -0.0018706083437909523, 
        -0.0019939289172219475, 
        -0.002534827533934738, 
        -0.0009737662823977987,
        -0.001822306074344211, 
        0.02273722712642639,
        -0.0004926714055603228,
        -0.003004986193713176
    ],
    [
        -0.0010044813810742047,
        -0.006671291663363668, 
        -0.00026003112503520596,
        -0.0005847470854143117,
        -0.000610813803292534, 
        -0.0017006155464219198, 
        -0.0018127292794043132, 
        -0.002304473368792744, 
        -0.0008852746134293082,
        -0.001656702778353048,
        -0.0004926714055603228, 
        0.020715738092779736,
        -0.0027319060426381574
    ],
    [
        -0.006126705645798535,
        -0.04069068981148138, 
        -0.0015860265723719278,
        -0.003566590020554541,
        -0.0037255806306354905,
        -0.010372686906803137, 
        -0.011056510274539641, 
        -0.014055840421926011,
        -0.005399619220795473,
        -0.010104846597247093,
        -0.003004986193713176,
        -0.0027319060426381574,
        0.11242198833850454
    ]
]

def softmax(x):
    """
    x - vector of n elements - input

    returns vector of n elements - softmax output
    """
    exp = np.exp(x)
    return exp / np.sum(exp, axis=0)

def dsoftmax_dx(x):
    """
    x - vector of n elements - input

    returns matrix n x n
    """
    n = len(x)
    x_softmax = softmax(x)
    grad = np.zeros(shape=(n, n))
    
    for i in range(n):
        for j in range(n):
            if i == j:
                grad[i, j] = x_softmax[j] * (1 - x_softmax[j])
            else:
                grad[i, j] = -x_softmax[i] * x_softmax[j]

    return grad


result = dsoftmax_dx(sample_input)

np.allclose(result, np.asarray(sample_output))

True

Напишите функцию, реализующую механизм внимания, как в [этой задаче](https://stepik.org/lesson/262248/step/10?unit=243131).

In [4]:
import numpy as np

sample_input = (
    [
        [
            0.3504305689198156,
            0.871844425624726,
            0.29345316540775357,
            0.49159320438393916,
            0.16391992930609034, 
            0.24589641847050037,
            0.34921020303336925,
            0.09968814035867879,
            0.8652385667745919,
            0.00906484385602968,
            0.6134586521086117,
            0.08104312584086149,
            0.643129733435556, 
            0.6610968673257929,
            0.6825169003800382
        ],
        [
            0.005641686042561211,
            0.3397733866278605,
            0.4408793722092307,
            0.6618752692611525,
            0.4192615374283991,
            0.6718589897811911, 
            0.23503584107912667,
            0.9972834040264165,
            0.6907780153811639,
            0.5160598448726361,
            0.4200243418824855,
            0.7745997472321381, 
            0.9124177261957108,
            0.627661131744206,
            0.7792319239076758
        ],
        [
            0.44947044223343224,
            0.39851993627332394,
            0.27645205987950927,
            0.3360502940952873, 
            0.20207394761469466,
            0.27730469648938627,
            0.9647449489128369,
            0.38480306917172535,
            0.7014748335636187, 
            0.5616919724157547,
            0.3082991954077743,
            0.43320540280287834,
            0.7682716834674514, 
            0.04669826413239708,
            0.7975639937877288
        ],
        [
            0.3529089999231677,
            0.5085801437940869,
            0.6686697864089949,
            0.9579051761714787,
            0.14893344048972235,
            0.6504801381978066,
            0.6554087909852483,
            0.2182290972559655,
            0.8874805349214916,
            0.8549111586624765,
            0.2256959432511686,
            0.7090739886051667, 
            0.9215898392742404,
            0.8361038069049278, 
            0.9807575571901945
        ],
        [
            0.93096165894251,
            0.21204415175966806,
            0.005393301311816812,
            0.6868163541395496, 
            0.17651743121260177,
            0.4276211882347165,
            0.7172630747046246, 
            0.6222321154458413,
            0.782866044726867,
            0.9401403417712956,
            0.6009251310321828,
            0.7689712777389628,
            0.011137370287858661,
            0.6750220130270511,
            0.3656918897133191
        ],
        [
            0.344423832576648,
            0.5200078573131781, 
            0.08090528060543856,
            0.6187002344784092,
            0.24428489996011238,
            0.18400539459399,
            0.40308101020726217,
            0.19255989698913867, 
            0.8010944590934469,
            0.20324438899818598, 
            0.4927144298170735, 
            0.03783988662477278,
            0.7705093963091103, 
            0.2520865403496373,
            0.40266725180440743
        ],
        [
            0.6681310493060861,
            0.2801674372250399, 
            0.6224648405839522,
            0.6287746784150413,
            0.864080498689899, 
            0.23833127705610258,
            0.20311743810136906,
            0.6646132937899738,
            0.23575457417289924,
            0.1869625695994146,
            0.7712148738157554, 
            0.15237041670323637,
            0.2763150373902683, 
            0.46500408886101585, 
            0.991468614310106
        ],
        [
            0.47955269815875845,
            0.18371674117676162,
            0.4749895427072034, 
            0.5127159626377625,
            0.14327300286458633,
            0.5921086963579639,
            0.21467664382766927,
            0.08984875049424312,
            0.5619088573772313, 
            0.6324525220346037, 
            0.65145500723789,
            0.5118736033583858,
            0.3791794826772541,
            0.7062193547285907,
            0.12888775429739185
        ],
        [
            0.8936198110390413,
            0.1499351596848777,
            0.23230300209801535,
            0.6275970485906217, 
            0.14179412142521963,
            0.44590423506527455,
            0.6398118989481705,
            0.44025473834142315,
            0.9690917160909921, 
            0.7329911430579539, 
            0.4723409208689966,
            0.30051845327308735,
            0.6517065287372249,
            0.11682964074366375,
            0.3356912564688531
        ],
        [
            0.5819483213886298,
            0.12715730125007862,
            0.6624081011547434,
            0.27210122174739293,
            0.3321414469174103,
            0.6738692045564213,
            0.49979407643990537,
            0.923095453187033, 
            0.8688108133354637,
            0.29672803800781,
            0.8422355709858387,
            0.22967466587429908,
            0.48606633908371566, 
            0.3302629931498261,
            0.9271715208940098
        ],
        [
            0.814229644181095,
            0.6269508015754929, 
            0.19067116118180638,
            0.6597416333912787,
            0.3042396495694798,
            0.5349586078017191,
            0.9889297007928726,
            0.5059647631701082,
            0.6586214714303461, 
            0.19972197385065704,
            0.730120041302739,
            0.9254129585548022,
            0.7774768791337286,
            0.5880525770183761, 
            0.4404909426586451
        ],
        [
            0.8393608070151912,
            0.551470751477307,
            0.3776646281929925,
            0.7403545806778788,
            0.01464073752506101,
            0.49682079457661743,
            0.12829037985166736,
            0.8323709714882789,
            0.4861583628986299, 
            0.10966510942571872,
            0.36384711262095637,
            0.008343156485128067,
            0.05481969871494197,
            0.11036480291979456,
            0.3495717657917
        ],
        [
            0.575668909069271,
            0.1948209406820347,
            0.5066632418120769,
            0.5610866065811511,
            0.7503051258152065,
            0.20250301475454058,
            0.9387177222181186,
            0.4214964558859865, 
            0.2441688535705906, 
            0.2852282954051667,
            0.7185375048873539,
            0.09961745251862686,
            0.507873295740294,
            0.9713796833363287,
            0.8218946227484244
        ],
        [
            0.14519733396011691,
            0.07264015089790021,
            0.7254237309701331,
            0.7437297525624584,
            0.3465971472185204,
            0.6489212261982703,
            0.2152569561085349,
            0.6476151760429897,
            0.2045187871395916,
            0.9599712380254137,
            0.28554199184758966,
            0.7701922251424572,
            0.7095119328780166, 
            0.7579558453415812,
            0.4251898428876446
        ]
    ],
    [
        0.30760147020407946, 
        0.1528992448227442,
        0.9387231083505163,
        0.12201982125460176,
        0.3159744925438269,
        0.555332538642272,
        0.8654043562058316,
        0.5523485724329922,
        0.6405492495162189,
        0.8421217300945876, 
        0.03415012932624606,
        0.0914780538557024,
        0.745151636966557,
        0.9885343010237021,
        0.02289480154711454
    ]
)

sample_output = (
    [
        0.4740581778159267, 0.34432834551260527, 0.4665167530554428, 0.6711507992179184, 0.30149325789082704,
        0.5216013512810533, 0.5739206841874827, 0.5011664230502018, 0.6659426824711602, 0.5870403517141657,
        0.48456045445664925, 0.5519092842133653, 0.6631513043916594, 0.6203158151313124, 0.6799614219857062
    ]
)

def softmax(z, axis=1):
    if axis == 1:
        exp_sum = np.sum(np.exp(z), axis=1).reshape(-1, 1)
    elif axis == 0:
        exp_sum = np.sum(np.exp(z), axis=0)
    else:
        exp_sum = np.sum(np.exp(z))
    return np.exp(z) / exp_sum

def attention(features, query):
    """
    features - InLen x EmbSize - features of elements of input sequence
    query - EmbSize - features of query object

    returns vector of size EmbSize - features, aggregated according to the query
    """
    features = np.asarray(features)
    query = np.asarray(query)
    
    unnorm_scores = features.dot(query)
    attention = softmax(unnorm_scores[np.newaxis, :])

    result = np.sum(features * attention.T, axis=0)
    return result

result = attention(*sample_input)
result

array([0.47405818, 0.34432835, 0.46651675, 0.6711508 , 0.30149326,
       0.52160135, 0.57392068, 0.50116642, 0.66594268, 0.58704035,
       0.48456045, 0.55190928, 0.6631513 , 0.62031582, 0.67996142])

Напишите функцию, реализующую механизм self-attention внимания с линейными преобразованиями, как в [этой задаче](https://stepik.org/lesson/262249/step/7?unit=243132).

In [11]:
import numpy as np

sample_input = (
    [
        [0.5175129778200084, 0.13021330700949507],
        [-0.3578609445921744, -0.07768163060380659],
        [-0.046577477754636734, -0.12288550821838619],
        [0.4424092505449793, -1.431399548551344],
        [0.753992222548331, -1.1210257338970167],
        [1.6736061037504428, -1.9789731491226337],
        [-1.4985152255486565, -1.6614802556117283],
        [-0.610065708073959, -0.8475335063027695],
        [-0.1657640783522184, -1.7079776825852762],
        [0.7857341373616981, 0.2956255012408635],
        [-0.49243028413686984, 0.01065675311085114],
        [0.20598401523943788, -0.6339670563637549],
        [-0.15698934123474126, 0.9516567843056503], 
        [-0.08965595798444444, 0.9923765422389716],
        [-0.9649404480809814, -0.6203623955866846]
    ],
    [
        [-0.4777373377067222, 1.384780896738418],
        [1.5537173245233542, -1.6151073640132454]
    ],
    [0.34068677065672126, -1.7225350645946451],
    [
        [1.667192089239799, 0.9072106203091014],
        [1.0017863742909645, -0.8578876449703756]
    ],
    [-0.3811843050970959, -0.15922065479353645],
    [
        [2.396375885002737, 0.23995979796695774],
        [0.21631803999882093, -0.6475173781192963]
    ],
    [1.448781271879823, -0.7144164516316529]
)

sample_output = (
    [
        [1.3828516277455327, -0.7052491362196253],
        [1.2990231347325174, 0.07313412990950195],
        [1.7291639000678214, 0.057097555962845575],
        [4.9977493686508465, 0.9611194260544597],
        [4.92077627317798, 0.9417305419570732],
        [5.027240226701748, 0.9675417617162317],
        [5.008405848246054, 0.9647146788718596],
        [4.617706670795201, 0.8915957294670733],
        [5.018586540911721, 0.966041694502124],
        [1.4121709865145842, -1.0082854213057408],
        [0.8082085153678329, -0.02863474439053916], 
        [4.261630070405553, 0.7878600291134995], 
        [1.1488305812076605, -1.1882802581002707], 
        [1.1929080669359462, -1.220148314614849],
        [3.9068303877684656, 0.7744398548226676]
    ]
)

def softmax(z, axis=1):
    if axis == 1:
        exp_sum = np.sum(np.exp(z), axis=1).reshape(-1, 1)
    elif axis == 0:
        exp_sum = np.sum(np.exp(z), axis=0)
    else:
        exp_sum = np.sum(np.exp(z))
    return np.exp(z) / exp_sum

def self_attention(features, proj_k, bias_k, proj_q, bias_q, proj_v, bias_v):
    """
    features - InLen x EmbSize - features of elements of input sequence
    proj_k - EmbSize x EmbSize - projection matrix to make keys from features
    bias_k - EmbSize - bias vector to make keys from features
    proj_q - EmbSize x EmbSize - projection matrix to make queries from features
    bias_q - EmbSize - bias vector to make queries from features
    proj_v - EmbSize x EmbSize - projection matrix to make values from features
    bias_v - EmbSize - bias vector to make values from features

    returns InLen x EmbSize
    """
    features = np.asarray(features)
    proj_k = np.asarray(proj_k)
    bias_k = np.asarray(bias_k)
    proj_q = np.asarray(proj_q)
    bias_q = np.asarray(bias_q)
    proj_v = np.asarray(proj_v)
    bias_v = np.asarray(bias_v)
    
    keys = features.dot(proj_k) + bias_k
    querries = features.dot(proj_q) + bias_q
    values = features.dot(proj_v) + bias_v
    
    logits = querries.dot(keys.T)
    attscores = softmax(logits, axis=1)

    result = attscores.dot(values)
    
    return result


result = self_attention(*sample_input)

result

array([[ 1.38285163, -0.70524914],
       [ 1.29902313,  0.07313413],
       [ 1.7291639 ,  0.05709756],
       [ 4.99774937,  0.96111943],
       [ 4.92077627,  0.94173054],
       [ 5.02724023,  0.96754176],
       [ 5.00840585,  0.96471468],
       [ 4.61770667,  0.89159573],
       [ 5.01858654,  0.96604169],
       [ 1.41217099, -1.00828542],
       [ 0.80820852, -0.02863474],
       [ 4.26163007,  0.78786003],
       [ 1.14883058, -1.18828026],
       [ 1.19290807, -1.22014831],
       [ 3.90683039,  0.77443985]])

## 5.6

Алгоритм машинного перевода переводит текст с английского языка на русский. Размер английского словаря равен 5000 токенов, русского - 8000. Оцените, сколько последовательностей рассмотрит декодер при генерации перевода наибольшей длины 5 при условии, что при декодировании используется алгоритм полного перебора?

В ответ запишите одно число.

Подсказка: в результате получится достаточно большое число.

In [1]:
8000**5

32768000000000000000

Алгоритм машинного перевода работает с переводом текста с английского языка на русский. Размер английского словаря равен 5000 токенов, русского - 8000. Оцените, сколько последовательностей сгенерирует декодер при переводе предложения длины 5 при условии, что используется лучевой поиск (beam search) при b=3 (b - ширина луча)? 

В ответ запишите одно число без пробелов.

In [1]:
3 ** 5

243

Пусть у нас есть предложение, состоящее из произвольных цифр. Например, '1576429830'.

Посчитайте перплексию этого предложения относительно модели, которая считает, что вероятность встретить каждую цифру равна 0.1.

 

*Напоминание: 
$PP(W) = P(w_1w_2..w_N)^{-1/N}$

In [1]:
(0.1**10)**(-1/10)

10.0

BLEU (bilingual evaluation understudy) - метрика для оценивания качества машинного перевода, основанная на сравнении перевода, предложенного алгоритмом, и референсного перевода (ground truth). Сравнение производится на основе подсчета n-грамм (n меняется от 1 до некоторого порога, например, 4), которые встретились и в предложенном переводе, и в референсном (ground truth). После подсчета совстречаемости n-грамм полученная метрика умножается на так называемый brevity penalty - штраф за слишком короткие варианты перевода. Brevity penalty считается как <количество слов в переводе, предложенном алгоритмом> / <количество слов в референсном переводе>.

Формула:
$BLEU = \text{brevity penalty} \cdot \left (\prod_{i=1}^n \text{precision}_i \right)^{1/n} \cdot 100\%$
, где $\text{brevity penalty} = min \left(1, \dfrac{\text{output length}}{\text{reference length}} \right)$

Пример:
![](https://ucarecdn.com/c3c754b4-29b3-4fec-ba6b-6e031b30de94/)


Задача

Посчитайте BLEU-score для следующего предложения. При подсчете метрики учитывайте n-граммы с $n \in [1,2,3]$

Перевод, предложенный алгоритмом: "Кошка вышла из дома и села на крыльцо"

Референсный перевод (ground truth): "Кошка вышла из комнаты и села на ступеньки"

 

Формат ответа: ответ запишите в виде процентов, округлив до целых.

In [20]:
import re

translation = "Кошка вышла из дома и села на крыльцо"
ground_truth = "Кошка вышла из комнаты и села на ступеньки"


def get_n_grams(sentence, n):
    sentence_splitted = re.findall(r'[\w]+', sentence)
    n_grams = []
    out_len = len(sentence_splitted) - n + 1
    for i in range(out_len):
        n_grams.append(sentence_splitted[i:i+n])
    return n_grams 

def precision(translation, ground_truth, n):
    n_grams_translation = get_n_grams(translation, n)
    n_grams_ground_truth = get_n_grams(ground_truth, n)

    intercetion = [value for value in n_grams_translation if value in n_grams_ground_truth]
    precision = len(intercetion) / len(n_grams_ground_truth)

    return precision

def bleu_score(translation, ground_truth, n_list):
    brevity_penalty = len(re.findall(r'[\w]+', translation)) / len(re.findall(r'[\w]+', ground_truth))
    
    precision_prod = 1
    for n in n_list:
        precision_prod *= precision(translation, ground_truth, n)
        
    return brevity_penalty * (precision_prod) ** (1/max(n_list)) * 100

bleu_score(translation, ground_truth, [1, 2, 3])

52.27579585747102